In [1]:
import dataiku
import pandas as pd
import numpy as np
import re
import country_converter as coco
import matplotlib.pyplot as plt
pd.options.display.max_colwidth = 100

Pre-processing for names involves standardization rules as per xls file taken as input for customer name and it's alias name and watch-list and it's alias name.

In [2]:
my_dataset = dataiku.Dataset("comb_data_ind_final3")
df = my_dataset.get_dataframe()


Columns (25,33,47,52,53,58,68) have mixed types.Specify dtype option on import or set low_memory=False.


In [4]:
df.head()

,N_PARTY_WATCHLIST_MATCH_ID,V_UCIF_ID,V_SCREENING_ID,N_WATCHLIST_PROFILE_ID,N_MATCH_SCORE,V_WATCHLIST_PARTY_NAME,V_INFO_SOURCE,V_WL_STATUS_CODE,D_DT_OF_SCREENING,V_COMMENTS,N_WORKFLOW_ID,F_RECENT_MATCH,V_RESOLUTION,V_REASON,F_DELTA_MATCH,D_CREATED_DTM,V_CREATED_BY,D_UPDATED_DTM,V_UPDATED_BY,N_SCREENING_VERSION,V_COUNTRY_OF_ONBOARDING,V_UCIF_ID_1,V_PARTY_CLASS,V_PARTY_TYPE,V_REGD_PARTY_NAME,V_ALIAS_NAME,V_GENDER,V_PARTY_FIRST_NAME,V_PARTY_MID_NAME,V_PARTY_LST_NAME,D_INCORPORATION_DATE,V_COUNTRY_OF_INCORP,V_COUNTRY_OF_BIRTH,V_PREVIOUS_NAME_IMMEDIATE,V_PREVIOUS_NAME_1,V_PREVIOUS_NAME_2,D_DOB,D_DECEASED_DATE,F_DECEASED_FLAG,V_COUNTRY_OF_BIRTH_1,V_FATHERS_NAME,V_MOTHERS_NAME,V_CREATED_BY_1,D_CREATED_DTM_1,V_UPDATED_BY_1,D_UPDATED_DTM_1,F_DELETE_FLAG,F_UPDATE_FLAG,V_LEGAL_STRUCTURE,F_PEP_STATUS,F_FMLY_PEP_STATUS,V_PEP_BIOGRAPHY,V_PEP_RELATIONSHIP,V_PEP_NAME,V_UCIF_ID_1_1,V_COUNTRY_OF_ONBOARDING_1,V_PARTY_GROUP,V_CITIZENSHIP,V_DUAL_CITIZENSHIP,V_OCC_CODE,V_COUNTRY_PERM_RES,V_SCREENING_STATUS,V_SCREENING_ID_1,V_CREATED_BY_PL,D_CREATED_DTM_PL,V_UPDATED_BY_PL,D_UPDATED_DTM_1_1,F_DELETE_FLAG_1,F_UPDATE_FLAG_1,N_SCREENING_VERSION_1,V_EMPLOYER_NAME,D_LAST_SCREENING_DATE,V_RACE,V_PARTY_CLASS_LEGACY,V_TITLE,V_RESIDENT_STATUS,V_UCIF_ID_2,AGG_UNQ_DOC_ID,AGG_V_ID_TYPE,V_PRIM_PTY_UCIF_ID,V_SEC_PTY_UCIF_ID,V_RELATION_TYPE_CD,N_NUMBER_OF_SHARES,N_SHAREHOLDING_PERCENT,V_SEC_PARTY_NAME,N_WATCHLIST_PROFILE_ID_1,V_WATCHLIST_PROFILE_UID,V_PROFILE_TYPE,V_PROFILE_NAME,V_ALIASES,V_TITLE_1,V_DOB,V_DOI,V_NATIONALITY,V_COI,V_IDENTIFICATION_NO,V_IC_NO,V_PLACE_OF_BIRTH,V_GENDER_WL,V_PEP_POSITION,V_BIOGRAPHY,V_INFO_SOURCE_1,V_REMARK,V_WL_STATUS_CODE_1,V_STATUS,D_CREATED_DTM_2,D_UPDATED_DTM_2,V_CATEGORY,V_SUB_CATEGORY,V_LOCATION_DETAILS,V_AGE,V_AGE_AS_OF_DATE
0,766713,83874871,14459134,4744768,100.0,MOHD BAKKE BIN SALLEH,PEPIN,81,2020-09-07 00:00:00,Alert has been closed as ‘No Hit’ in FICO system previously.,263907.0,N,FP,UI,N,2020-09-07 16:23:35,c8737064-b5aa-4a35-9930-73b211ea5137,2021-06-06,80001598,1,MYS,83874871,I,12000.0,MOHD BAKKE BIN SALLEH,NaN,M,NaN,NaN,NaN,NaN,NaN,DD,NaN,NaN,NaN,9999-01-01 00:00:00,NaN,N,DD,NaN,NaN,CIFAD208,2002-12-12 21:02:51,NaN,1900-01-01 00:00:00,N,NaN,INDV,NaN,NaN,NaN,NaN,NaN,83874871,MYS,BUM,MYS,NaN,DD,MYS,NOHITS,14459134,CIFAD208,2002-12-12 21:02:51,80006967,2023-04-12 10:16:38.331212,N,NaN,1,NaN,2021-08-04 00:00:00,MYS,500.0,MRR,CTZN,83874871.0,"[""NOT/APPL""]","[""OLIC""]",NaN,NaN,NaN,NaN,NaN,NaN,4744768,FP00818,I,MOHD BAKKE BIN SALLEH,NaN,Tan Sri,13/07/1954,NaN,MALAYSIA,NaN,4630804,5.407131e+11,NaN,M,Senior Official - SOE,Member of the Board of Directors of Petroliam Nasional Berhad (SOE) (reported Aug 2019 - ). Chai...,PEPIN,POLITICALLY EXPOSED PERSON,81,A,2020-09-04 22:51:06.04282,NaN,NaN,NaN,NaN,NaN,NaN
1,662693,65805657,9619904,4754736,95.0,MOHD JEFFRI BIN ABDULLAH,ICL,75,2020-09-07 00:00:00,"Mismatch - NRIC, DOB\nCustomer : 790913145669\nWatch person : 790710146127",1518490.0,N,FP,PI,N,2020-09-07 13:59:59,33b05a8f-7262-4c1c-a1af-714d6b1eb38e,2021-05-21,9344,1,MYS,65805657,I,12000.0,JEFRI BIN MOHD ABDULLAH,NaN,M,NaN,NaN,NaN,NaN,NaN,DD,NaN,NaN,NaN,1979-09-13 00:00:00,NaN,N,DD,NaN,NaN,0006885A,2010-07-29 15:06:58,TRILLIUM,2011-01-15 04:12:11,N,NaN,INDV,NaN,NaN,NaN,NaN,NaN,65805657,MYS,BUM,MYS,NaN,9310,MYS,NOHITS,9619904,0006885A,2010-07-29 15:06:58,00009344,2021-05-21 17:42:53.896309,N,NaN,1,PJ DEVELOPMENT HOLDINGS BERHAD,2020-09-07 00:00:00,MYS,500.0,MRR,CTZN,65805657.0,"[""790913145669""]","[""NWIC""]",NaN,NaN,NaN,NaN,NaN,NaN,4754736,FC556,I,MOHD JEFFRI BIN ABDULLAH,NaN,NaN,10/07/1979,NaN,MALAYSIA,NaN,NaN,7.907101e+11,NaN,M,NaN,REQUIRES ECDD & SENIOR MGMT APPROVAL,ICL,REQUIRES ECDD & SENIOR MGMT APPROVAL,75,A,2020-09-04 22:51:06.04282,NaN,NaN,NaN,NaN,NaN,NaN
2,370521,11748595,5321381,4754736,95.0,MOHD JEFFRI BIN ABDULLAH,ICL,75,2020-09-07 00:00:00,NaN,135647.0,N,FP,PI,N,2020-09-07 07:33:32,1c5e7e75-6e6d-474a-b393-8d2017f12ecf,2020-10-02,136544,1,MYS,11748595,I,12000.0,MOHD JEFRI BIN ABDULLAH,NaN,M,NaN,NaN,NaN,NaN,NaN,DD,

In [8]:
df.shape

(701181, 112)

In [9]:
df.columns

Index(['N_PARTY_WATCHLIST_MATCH_ID', 'V_UCIF_ID', 'V_SCREENING_ID', 'N_WATCHLIST_PROFILE_ID', 'N_MATCH_SCORE', 'V_WATCHLIST_PARTY_NAME', 'V_INFO_SOURCE', 'V_WL_STATUS_CODE', 'D_DT_OF_SCREENING', 'V_COMMENTS',
       ...
       'V_REMARK', 'V_WL_STATUS_CODE_1', 'V_STATUS', 'D_CREATED_DTM_2', 'D_UPDATED_DTM_2', 'V_CATEGORY', 'V_SUB_CATEGORY', 'V_LOCATION_DETAILS', 'V_AGE', 'V_AGE_AS_OF_DATE'], dtype='object', length=112)

# standardization rules

In [7]:
my_dataset = dataiku.Dataset("Standradization_rules_draft__base_line_model")
scr_rules = my_dataset.get_dataframe()
print(scr_rules.loc[45:51,"V_SCREENING_VALUE_1"])

45                                               \.
46      MUHAMMAD |MUHAMMED |MUHAMAD |MUHAMED |MUHD 
47     MOHAMMAD |MOHAMMED |MOHAMAD |MOHAMED | MOHD 
48    ABDUL |ABDOUL |ABDUR |ABDOL |ABDOOL |ABD |AB 
49                            NOOR |NOR |NUUR |NUR 
50                             AHMAD |AHMED |AHMET 
51                              ABDULLAH |ABDOLLAH 
Name: V_SCREENING_VALUE_1, dtype: object


In [16]:
scr_rules

,Sequence of the rules,col_1,col_2,V_SCREENING_VALUE_1,Remarks,Solution,V_SCREENING_VALUE_2,col_7
0,1,Patronym,NaN,A/L | A/P | S/O | D/O | SO | DO | AP,Terms are used to signify the father's name like A/P refers to Daughter of. For eg. VERNISHA J...,We remove these these terms,NaN,NaN
1,1,Patronym,NaN,BINTI | BINTE | B | BTE | BT | IBNI | IBNU | ANAK,Son / daughter of. for Malay & indegenous people. For eg. FARIDAH BINTE MAHMUD,We remove these these terms,NaN,NaN
2,2,Apostrophe,NaN,\',Change in sound,Remove it,NaN,NaN
3,3,Hyphen,NaN,[-],use to join words,Replace with white space character,NaN,NaN
4,4,Prefix,NaN,HAJI |HAJJAH |HAJAH |HJH |HJ |ALHAJ |HADJI,Tilte for˜Muslim˜men and women who have performed the˜Hajj (pilgrimage). That means these terms ...,Remove the terms if present in Individual( also in father's name if it comes),NaN,NaN
5,4,Prefix,NaN,WIRA |PANGLIMA |IR |DR. |PROF. |SR,Other salutations; professional rank,Remove the salutations; professional rank,NaN,NaN
6,5,Titles for Royal and honoured people,NaN,K.D.Y.T.M. |K.D.Y.M.M. |D.Y.M.M. |D.Y.T.M. |Y.T.M. |D.Y.A.M. |Y.M.M. |Y.A.M. |Y.BHG. |KDYTM |KDY...,Abbreviation for the king title,Remove the title,NaN,NaN
7,5,Titles for Royal and honoured people,NaN,YANG DI PERTUAN AGONG |DULI YANG MAHA MULIA SERI PADUKA BAGINDA,Official title of the king(ruler) of all Malaysia,Remove the title,NaN,NaN
8,5,Titles for Royal and honoured people,NaN,TIMBALAN YANG DI PERTUAN AGONG,NaN,Remove the title,NaN,NaN
9,5,Titles for Royal and honoured people,NaN,YANG DI PERTUAN BESAR |YANG DI PERTUA NEGERI |SULTAN,NaN,Remove the title,NaN,NaN


In [3]:
my_dataset = dataiku.Dataset("Standradization_rules_draft__base_line_model")
scr_rules = my_dataset.get_dataframe()
print(scr_rules.loc[45:51,"V_SCREENING_VALUE_1"])
scr_rules.loc[46:51,"V_SCREENING_VALUE_1"]=scr_rules.loc[46:51,"V_SCREENING_VALUE_1"].apply(lambda x: x.lstrip("( ").rstrip(")"))

scr_rules_2=scr_rules.iloc[21:46,:] # Punctuation marks
scr_rules_3=scr_rules.iloc[46:53,:]  #  Standardization rules
scr_rules=scr_rules.iloc[0:16,:]    # patronym & honourary titles




45                                               \.
46      MUHAMMAD |MUHAMMED |MUHAMAD |MUHAMED |MUHD 
47     MOHAMMAD |MOHAMMED |MOHAMAD |MOHAMED | MOHD 
48    ABDUL |ABDOUL |ABDUR |ABDOL |ABDOOL |ABD |AB 
49                            NOOR |NOR |NUUR |NUR 
50                             AHMAD |AHMED |AHMET 
51                              ABDULLAH |ABDOLLAH 
Name: V_SCREENING_VALUE_1, dtype: object


In [172]:
new_df=df.copy()

In [173]:
df["V_COMMENTS"]=df["V_COMMENTS"].apply(lambda x: '' if pd.isna(x) else x)

In [174]:
# Our objective of validation would be like for TP score should be greater threshold & for FP scoe should be less than threshold

# Customer Name 

In [175]:
# imputing all names & then applying std rules on names
# imputation from df[[ 'V_PREVIOUS_NAME_IMMEDIATE', 'V_PREVIOUS_NAME_1','V_PREVIOUS_NAME_2','V_PARTY_FIRST_NAME','V_PARTY_MID_NAME','V_PARTY_LST_NAME']]


In [176]:
# datatypes present in array
print(df['V_PREVIOUS_NAME_1'].apply(lambda x: type(x)).unique())
print(df['V_PREVIOUS_NAME_2'].apply(lambda x: type(x)).unique())

[<class 'float'>]
[<class 'float'>]


In [177]:
df['V_PREVIOUS_NAME_1']=df['V_PREVIOUS_NAME_1'].apply(lambda x : [""]  if pd.isna(x) else [str(x).upper()])
df['V_PREVIOUS_NAME_2']=df['V_PREVIOUS_NAME_2'].apply(lambda x :  [""] if pd.isna(x) else [str(x).upper()])
df['V_PREVIOUS_NAME_IMMEDIATE']=df['V_PREVIOUS_NAME_IMMEDIATE'].apply(lambda x :  [""] if pd.isna(x) else [str(x).upper()])


In [178]:
# pre-processing on "V_REGD_PARTY_NAME"
df["V_REGD_PARTY_NAME"]=new_df["V_REGD_PARTY_NAME"].values
df["V_REGD_PARTY_NAME"]=df["V_REGD_PARTY_NAME"].apply(lambda x : [""]  if pd.isna(x) else [str(x).upper()])
# not more than element is present in V_REGD_PARTY_NAMES
df[df["V_REGD_PARTY_NAME"].apply(lambda x: len(x)>1)]

,N_PARTY_WATCHLIST_MATCH_ID,V_UCIF_ID,V_SCREENING_ID,N_WATCHLIST_PROFILE_ID,N_MATCH_SCORE,V_WATCHLIST_PARTY_NAME,V_INFO_SOURCE,V_WL_STATUS_CODE,D_DT_OF_SCREENING,V_COMMENTS,N_WORKFLOW_ID,F_RECENT_MATCH,V_RESOLUTION,V_REASON,F_DELTA_MATCH,D_CREATED_DTM,V_CREATED_BY,D_UPDATED_DTM,V_UPDATED_BY,N_SCREENING_VERSION,V_COUNTRY_OF_ONBOARDING,V_UCIF_ID_1,V_PARTY_CLASS,V_PARTY_TYPE,V_REGD_PARTY_NAME,V_ALIAS_NAME,V_GENDER,V_PARTY_FIRST_NAME,V_PARTY_MID_NAME,V_PARTY_LST_NAME,D_INCORPORATION_DATE,V_COUNTRY_OF_INCORP,V_COUNTRY_OF_BIRTH,V_PREVIOUS_NAME_IMMEDIATE,V_PREVIOUS_NAME_1,V_PREVIOUS_NAME_2,D_DOB,D_DECEASED_DATE,F_DECEASED_FLAG,V_COUNTRY_OF_BIRTH_1,V_FATHERS_NAME,V_MOTHERS_NAME,V_CREATED_BY_1,D_CREATED_DTM_1,V_UPDATED_BY_1,D_UPDATED_DTM_1,F_DELETE_FLAG,F_UPDATE_FLAG,V_LEGAL_STRUCTURE,F_PEP_STATUS,F_FMLY_PEP_STATUS,V_PEP_BIOGRAPHY,V_PEP_RELATIONSHIP,V_PEP_NAME,V_UCIF_ID_1_1,V_COUNTRY_OF_ONBOARDING_1,V_PARTY_GROUP,V_CITIZENSHIP,V_DUAL_CITIZENSHIP,V_OCC_CODE,V_COUNTRY_PERM_RES,V_SCREENING_STATUS,V_SCREENING_ID_1,V_CREATED_BY_PL,D_CREATED_DTM_PL,V_UPDATED_BY_PL,D_UPDATED_DTM_1_1,F_DELETE_FLAG_1,F_UPDATE_FLAG_1,N_SCREENING_VERSION_1,V_EMPLOYER_NAME,D_LAST_SCREENING_DATE,V_RACE,V_PARTY_CLASS_LEGACY,V_TITLE,V_RESIDENT_STATUS,V_UCIF_ID_2,AGG_UNQ_DOC_ID,AGG_V_ID_TYPE,V_PRIM_PTY_UCIF_ID,V_SEC_PTY_UCIF_ID,V_RELATION_TYPE_CD,N_NUMBER_OF_SHARES,N_SHAREHOLDING_PERCENT,V_REGD_PARTY_NAME_1,N_WATCHLIST_PROFILE_ID_1,V_WATCHLIST_PROFILE_UID,V_PROFILE_TYPE,V_PROFILE_NAME,V_ALIASES,V_TITLE_1,V_DOB,V_DOI,V_NATIONALITY,V_COI,V_IDENTIFICATION_NO,V_IC_NO,V_PLACE_OF_BIRTH,V_GENDER_1,V_PEP_POSITION,V_BIOGRAPHY,V_INFO_SOURCE_1,V_REMARK,V_WL_STATUS_CODE_1,V_STATUS,D_CREATED_DTM_2,D_UPDATED_DTM_2,V_CATEGORY,V_SUB_CATEGORY,V_LOCATION_DETAILS,V_AGE,V_AGE_AS_OF_DATE


In [179]:
# imputation done
df[["V_REGD_PARTY_NAME", 'V_PREVIOUS_NAME_IMMEDIATE', 'V_PREVIOUS_NAME_1','V_PREVIOUS_NAME_2']]
name_df=pd.DataFrame({})
name_df["ALL_PARTY_NAMES"]=df[["V_REGD_PARTY_NAME", 'V_PREVIOUS_NAME_IMMEDIATE', 'V_PREVIOUS_NAME_1','V_PREVIOUS_NAME_2']].apply(lambda x: x[0]+x[1]+x[2]+x[3],axis=1)

In [180]:
# Regd party names having HAJI
df[df["V_REGD_PARTY_NAME"].apply(lambda x: len(re.findall("HAJI",x[0]))>0)]["V_REGD_PARTY_NAME"]

607         [ABDUL RASHID HAJI ABDUL MAJID]
672                   [ABD RAHMAN BIN HAJI]
1031             [NURUL HAJIDAH BINTI ALWI]
1126      [AHMAD MUHAJIR BIN MUZAFFAR SHAH]
1952           [MOHD FAUZAN BIN HAJI AHMAD]
                        ...                
713519        [FARIDAH BINTI HAJI ABDULLAH]
713619           [JAMALUDIN BIN HAJI OSMAN]
714275            [SANIAH BINTI HAJI AWANG]
715050               [AHMAD BIN HAJI BAKAR]
715146            [ABDULLAH HAJI ABD MAJID]
Name: V_REGD_PARTY_NAME, Length: 1248, dtype: object

In [181]:
df[df["V_REGD_PARTY_NAME"].apply(lambda x: len(re.findall(" A/L | A/P| S/O | D/O ",x[0]))>0)]["V_REGD_PARTY_NAME"]

12        [GANESAN S/O SUBRAMANIAM]
13        [GANESAN A/L SUBRAMANIAM]
14        [GANESAN A/L SUBRAMANIAM]
15        [GANESON A/L SUBRAMANIAM]
16        [GANESAN A/L SUBRAMANIAM]
                    ...            
714089      [SUGANTHI A/P CHANDRAN]
714340      [KUBENDRAN A/L MUNIANY]
714456      [GANESON A/L SHANMUGAM]
714755      [GOBI A/L VIJAYA RAJAH]
715215        [PAVITHRA A/P SEGREN]
Name: V_REGD_PARTY_NAME, Length: 10619, dtype: object

In [182]:
df[["V_REGD_PARTY_NAME", 'V_PREVIOUS_NAME_IMMEDIATE', 'V_PREVIOUS_NAME_1','V_PREVIOUS_NAME_2']]

,V_REGD_PARTY_NAME,V_PREVIOUS_NAME_IMMEDIATE,V_PREVIOUS_NAME_1,V_PREVIOUS_NAME_2
0,[SAHARUDDIN BIN MD SALLEH],[],[],[]
1,[HO KIONG CHAN],[],[],[]
2,[AHMAD ASABI BIN ISMAIL],[],[],[]
3,[ANG LEE ENG],[],[],[]
4,[ENG LEE ANG],[],[],[]
...,...,...,...,...
715256,[MUHAMMAD UMAR],[],[],[]
715257,[MUHAMMAD UMAR],[],[],[]
715258,[MUHAMMAD UMAIR],[],[],[]
715259,[MAIKO SUZUKI],[],[],[]


In [183]:
df[df["V_REGD_PARTY_NAME"].apply(lambda x: len(re.findall("\'",x[0]))>0)]["V_REGD_PARTY_NAME"]

241                       [KHALID BIN JA'AFAR]
414        [MOHAMMAD FAZRIN HELMI BIN JA'AFAR]
965                  [NUR'AINI BINTI ABDULLAH]
1052                  [NUR'ATIQAH BINTI AHMAD]
1577      [MOHD NURIZZUDDIN AZRA'AI BIN AWANG]
                          ...                 
713241                     [O'NEIL XAVIER HAN]
713284             [MOHD ZA'IM BIN MAT YUSOFF]
713286              [MOHD 'AZIM BIN MAT YUSOF]
713295            [MUHAMMAD 'IZZAT BIN HASSAN]
713310                        [H'NG YEAT KOOI]
Name: V_REGD_PARTY_NAME, Length: 1532, dtype: object

In [184]:
# presence of hyphen
df[df["V_REGD_PARTY_NAME"].apply(lambda x:len(re.findall("-",x[0]))>0)]["V_REGD_PARTY_NAME"]

310       [MUAAMAR ABDUL WAHAB MELHI AL-ASWAD]
2563                         [CHO HYUN - JOON]
2578                           [CHEN SHIH-LUN]
4182           [ZUR-AINNATASHA BINTI ZULKIFLI]
4824                             [CHEN MEI-YU]
                          ...                 
712009                         [CHANG YU-LING]
712109                       [ZAIN-UL-ABIDEEN]
712247                            [TAN E-LYNN]
713163                           [LEE SO-YONG]
713308                          [LIN, YU-CHEN]
Name: V_REGD_PARTY_NAME, Length: 933, dtype: object

In [185]:
name_df["ALL_PARTY_NAMES"].head()

0    [SAHARUDDIN BIN MD SALLEH, , , ]
1               [HO KIONG CHAN, , , ]
2      [AHMAD ASABI BIN ISMAIL, , , ]
3                 [ANG LEE ENG, , , ]
4                 [ENG LEE ANG, , , ]
Name: ALL_PARTY_NAMES, dtype: object

In [186]:
# Patronym , ', - , Honourary Titles 
for k in [' A/L | A/P | S/O | D/O | SO | DO | AP ',' BINTI | BINTE | B | BTE | BT | IBNI | IBNU | ANAK ']:  # if any patronym title comes , then remove it 
    name_df["ALL_PARTY_NAMES"]=name_df["ALL_PARTY_NAMES"].apply(lambda x: [re.sub(k," ",x[0]),re.sub(k," ",x[1]),re.sub(k," ",x[2]),re.sub(k," ",x[3])])

#' apostrophe Abd Alfattah So'ud Abd Alfattah DARDAS
name_df["ALL_PARTY_NAMES"]=name_df["ALL_PARTY_NAMES"].apply(lambda x: [re.sub("\'","",x[0]),re.sub("\'","",x[1]),re.sub("\'","",x[2]),re.sub("\'","",x[3])])

# - dash
name_df["ALL_PARTY_NAMES"]=name_df["ALL_PARTY_NAMES"].apply(lambda x: [re.sub("-"," ",x[0]),re.sub("-"," ",x[1]),re.sub("-"," ",x[2]),re.sub("-"," ",x[3])])

#Honourary Titles 
for k in scr_rules["V_SCREENING_VALUE_1"].values[4:16].tolist():
    print(k)
    name_df["ALL_PARTY_NAMES"]=name_df["ALL_PARTY_NAMES"].apply(lambda x: [re.sub("^("+k+")","",x[0]),re.sub("^("+k+")","",x[1]),re.sub("^("+k+")","",x[2]),re.sub("^("+k+")","",x[3])]) 
# We need to remove HAJI & all such terms if in case they come in b/w names.
name_df["ALL_PARTY_NAMES"]=name_df["ALL_PARTY_NAMES"].apply(lambda x: [re.sub(" HAJI | HAJJAH | HAJAH | HJH | HJ | ALHAJ | HADJI "," ",i) for i in x])
name_df["ALL_PARTY_NAMES"]=name_df["ALL_PARTY_NAMES"].apply(lambda x: [re.sub("( HAJI| HAJJAH| HAJAH| HJH| HJ| ALHAJ| HADJI)$","",i) for i in x])


        # Need to add a cap ^ to remove only the names where titles come as prefix.

HAJI |HAJJAH |HAJAH |HJH |HJ |ALHAJ |HADJI 
WIRA |PANGLIMA |IR |DR. |PROF. |SR 
K.D.Y.T.M. |K.D.Y.M.M. |D.Y.M.M. |D.Y.T.M. |Y.T.M. |D.Y.A.M. |Y.M.M. |Y.A.M. |Y.BHG. |KDYTM |KDYMM |DYMM |DYTM |DYAM |YMM |YTM 
YANG  DI PERTUAN  AGONG |DULI  YANG  MAHA  MULIA  SERI  PADUKA  BAGINDA 
TIMBALAN  YANG  DI PERTUAN  AGONG 
YANG DI PERTUAN BESAR |YANG  DI PERTUA  NEGERI  |SULTAN 
RAJA  PERMAISURI  AGONG 
TUNKU  AMPUAN  BESAR |TUNKU  AMPUAN |TUNKU  PUAN  BESAR |SULTANAH |TENGKU  AMPUAN  BESAR |TENGKU  AMPUAN  MAHKOTA |TENGKU  AMPUAN | TENGKU  BESAR |TENGKU  MAHKOTA |PADUKA  BONDA  RAJA 
RAJA  PERMAISURI |PERMAISURI |RAJA  MUDA |RAJA  PUAN  MUDA |RAJA  PUAN  BESAR |RAJA  DI HILIR |TENGKU  PUAN  MUDA |TENGKU  PUAN |YANG  DI-PERTUAN  MUDA |YANG  DI-PERTUAN  BESAR 
TUANKU |TUNKU |TUN 
RAJA |TENGKU |MEK |NIK |TUAN |WAN |PUTERI |PUTRI |AWANG |ABANG |DAYANG |CIK  PUAN  BESAR |CIK  PUAN |ENCHE  BESAR |CHE  PUAN  BESAR |SYED |SY |SAYED |SHARIFAH |SH |SYARIFAH 
TAN  SRI |DATUK  SERI |DATUK  SRI |DATUK |DAT

In [187]:
name_df["ALL_PARTY_NAMES"]

0         [SAHARUDDIN BIN MD SALLEH, , , ]
1                    [HO KIONG CHAN, , , ]
2           [AHMAD ASABI BIN ISMAIL, , , ]
3                      [ANG LEE ENG, , , ]
4                      [ENG LEE ANG, , , ]
                        ...               
715256               [MUHAMMAD UMAR, , , ]
715257               [MUHAMMAD UMAR, , , ]
715258              [MUHAMMAD UMAIR, , , ]
715259                [MAIKO SUZUKI, , , ]
715260                [LU, YUANYUAN, , , ]
Name: ALL_PARTY_NAMES, Length: 715261, dtype: object

In [188]:
df[df["V_REGD_PARTY_NAME"].apply(lambda x: len(re.findall("@",x[0]))>0)]["V_REGD_PARTY_NAME"]

460         [AMINUDDIN BIN MD SOM @ ABDULLAH]
784           [NORAINI BINTI IBRAHIM @ AWANG]
902          [RAMARAO@JANAGIRAM A/L RAMASAMY]
989              [NG SEH HOW @ NG KHAI SIONG]
1010         [BEH TEK LOONG @ MAH PENG CHEAH]
                         ...                 
712097                [NAZRI BIN SANTA @ EDD]
713195       [MOHD NAFEE @ NAFIEE BIN KASSIM]
714084         [CHONG VUI CHUNG @ JOHN CHONG]
714546     [MOHD RIDWAN BIN ABDULLAH @ AWANG]
715103    [MOHD YADEY BIN IDRIS @ MOHD IDRIS]
Name: V_REGD_PARTY_NAME, Length: 1477, dtype: object

In [189]:
df[df["V_REGD_PARTY_NAME"].apply(lambda x: sum([1 if len(re.findall("\(.+\)",x[i]))>0 else 0 for i in range(len(x))])>0)]["V_REGD_PARTY_NAME"]

<>:1: DeprecationWarning: invalid escape sequence \(
<>:1: DeprecationWarning: invalid escape sequence \(
<ipython-input-189-65cffc9fdc40>:1: DeprecationWarning: invalid escape sequence \(
  df[df["V_REGD_PARTY_NAME"].apply(lambda x: sum([1 if len(re.findall("\(.+\)",x[i]))>0 else 0 for i in range(len(x))])>0)]["V_REGD_PARTY_NAME"]


46777                [SOON (SOIN) CHIN WEI]
77506                 [NGE (NG) JIAN XIANG]
132534                  [LEE WAN TIN (DR.)]
155393        [FATIMAH (HJH) BT HJ IBRAHIM]
170299         [YEO KAY HIAN (YANG QIXIAN)]
191992          [WAN IBRAHIM WAN HASSAN(A)]
197077            [MOHAMAD BIN MUSA (HAJI)]
319555    [MOHAMMAD REZUANUL HOQUE (JEWEL)]
341464            [MOHAMAD BIN MUSA (HAJI)]
457490        [SEAH KOK PENG (SHE GUOPING)]
490049            [MOHAMAD BIN MUSA (HAJI)]
516227          [ZAIPAH BINTI YUSOF (MOHD)]
526406    [SEOW CHOON CHOON(XIAO CHUNCHUN)]
613951                  [LEE WAN TIN (DR.)]
682858                  [LEE WAN TIN (DR.)]
689932             [OOI YEOW TATT(BERNARD)]
Name: V_REGD_PARTY_NAME, dtype: object

In [190]:
# Pre-processing (Step-2) & getting Alias names
# () Alias name  -  below alias name 2 column, has list in each row.
name_df["V_ALIAS_NAME_2"]=name_df["ALL_PARTY_NAMES"].apply(lambda x: [re.findall("\(.+\)",x[i]) for i in range(len(x))])
                                                                

name_df["V_ALIAS_NAME_2"]=name_df["V_ALIAS_NAME_2"].apply(lambda x: [x[i][0].lstrip("(").rstrip(")") if len(x[i])>0 else "" for i in range(len(x))])
 
name_df["ALL_PARTY_NAMES"]=name_df["ALL_PARTY_NAMES"].apply(lambda x: [re.sub(" \(.+\)","",x[i]) for i in range(len(x))])


#@ 
name_df["V_ALIAS_NAME_3"]=[""]*(df.shape[0])
name_df["ALL_PARTY_NAMES"]=name_df["ALL_PARTY_NAMES"].apply(lambda x: [re.split("@",x[i]) for i in range(len(x))])

name_df["V_ALIAS_NAME_3"]=name_df["ALL_PARTY_NAMES"].apply(lambda x: [x[i][1].lstrip(" ") if len(x[i]) >1 else "" for i in range(len(x))])

name_df["ALL_PARTY_NAMES"]=name_df["ALL_PARTY_NAMES"].apply( lambda x: [x[i][0].rstrip(" ") for i in range(len(x)) ])
                                                                    


# slash - if more than one slash than issue shall arise
name_df["V_ALIAS_NAME_4"]=name_df["ALL_PARTY_NAMES"].apply(lambda x: [re.split(" / ",x[i]) for i in range(len(x))])
name_df["ALL_PARTY_NAMES"]=name_df["V_ALIAS_NAME_4"].apply(lambda x:[x[i][0] for i in range(len(x))])
name_df["V_ALIAS_NAME_4"]=name_df["V_ALIAS_NAME_4"].apply(lambda x:[x[i][1] if len(x[i])>1 else "" for i in range(len(x))])


<>:3: DeprecationWarning: invalid escape sequence \(
<>:8: DeprecationWarning: invalid escape sequence \(
<>:3: DeprecationWarning: invalid escape sequence \(
<>:8: DeprecationWarning: invalid escape sequence \(
<ipython-input-190-350b69516c74>:3: DeprecationWarning: invalid escape sequence \(
  name_df["V_ALIAS_NAME_2"]=name_df["ALL_PARTY_NAMES"].apply(lambda x: [re.findall("\(.+\)",x[i]) for i in range(len(x))])
<ipython-input-190-350b69516c74>:8: DeprecationWarning: invalid escape sequence \(
  name_df["ALL_PARTY_NAMES"]=name_df["ALL_PARTY_NAMES"].apply(lambda x: [re.sub(" \(.+\)","",x[i]) for i in range(len(x))])


In [191]:
df["V_REGD_PARTY_NAME"]=df["V_REGD_PARTY_NAME"].apply(lambda x: x[0])
print(df[df["V_REGD_PARTY_NAME"].str.contains("[Á,Ä,Â,Ã]")]["V_REGD_PARTY_NAME"])
print(df[df["V_REGD_PARTY_NAME"].str.contains("[Å]")]["V_REGD_PARTY_NAME"])
print(df[df["V_REGD_PARTY_NAME"].str.contains("[É,Ë,Ê,È]")]["V_REGD_PARTY_NAME"])


2403      ZHANG, HONGYING
2407          ZHAN, PUWEN
2415             LIU, JIA
2570          ZHANG, MENG
2572          ZHANG, PENG
               ...       
715208         WANG, QIAN
715214      LI, XIANGGANG
715221         ZHANG, YAN
715223        ZHANG, YING
715260       LU, YUANYUAN
Name: V_REGD_PARTY_NAME, Length: 6126, dtype: object
Series([], Name: V_REGD_PARTY_NAME, dtype: object)
2403      ZHANG, HONGYING
2407          ZHAN, PUWEN
2415             LIU, JIA
2570          ZHANG, MENG
2572          ZHANG, PENG
               ...       
715208         WANG, QIAN
715214      LI, XIANGGANG
715221         ZHANG, YAN
715223        ZHANG, YING
715260       LU, YUANYUAN
Name: V_REGD_PARTY_NAME, Length: 6126, dtype: object


In [192]:
# Accented Characters
name_df["ALL_PARTY_NAMES"]=name_df["ALL_PARTY_NAMES"].apply(lambda x: [re.sub("[Á,Ä,Â,Ã]","A",x[0]),re.sub("[Á,Ä,Â,Ã]","A",x[1]),re.sub("[Á,Ä,Â,Ã]","A",x[2]),re.sub("[Á,Ä,Â,Ã]","A",x[3])])
name_df["ALL_PARTY_NAMES"]=name_df["ALL_PARTY_NAMES"].apply(lambda x: [re.sub("[Å]","AA",x[0]),re.sub("[Å]","AA",x[1]),re.sub("[Å]","AA",x[2]),re.sub("[Å]","AA",x[3])])
name_df["ALL_PARTY_NAMES"]=name_df["ALL_PARTY_NAMES"].apply(lambda x: [re.sub("[É,Ë,Ê,È]","E",x[0]),re.sub("[É,Ë,Ê,È]","E",x[1]),re.sub("[É,Ë,Ê,È]","E",x[2]),re.sub("[É,Ë,Ê,È]","E",x[3])])
name_df["ALL_PARTY_NAMES"]=name_df["ALL_PARTY_NAMES"].apply(lambda x: [re.sub("[Í,Ï,Î]","I",x[0]),re.sub("[Í,Ï,Î]","I",x[1]),re.sub("[Í,Ï,Î]","I",x[2]),re.sub("[Í,Ï,Î]","I",x[3])])
name_df["ALL_PARTY_NAMES"]=name_df["ALL_PARTY_NAMES"].apply(lambda x: [re.sub("[Ó,Ô,Ö,Õ]","O",x[0]),re.sub("[Ó,Ô,Ö,Õ]","O",x[1]),re.sub("[Ó,Ô,Ö,Õ]","O",x[2]),re.sub("[Ó,Ô,Ö,Õ]","O",x[3])])
name_df["ALL_PARTY_NAMES"]=name_df["ALL_PARTY_NAMES"].apply(lambda x: [re.sub("[Ú,Ü,Ù]","U",x[0]),re.sub("[Ú,Ü,Ù]","U",x[1]),re.sub("[Ú,Ü,Ù]","U",x[2]),re.sub("[Ú,Ü,Ù]","U",x[3])])

name_df["ALL_PARTY_NAMES"]=name_df["ALL_PARTY_NAMES"].apply(lambda x: [re.sub("[Ø]","OE",x[0]),re.sub("[Ø]","OE",x[1]),re.sub("[Ø]","OE",x[2]),re.sub("[Ø]","OE",x[3])])
name_df["ALL_PARTY_NAMES"]=name_df["ALL_PARTY_NAMES"].apply(lambda x: [re.sub("[Ç]","C",x[0]),re.sub("[Ç]","C",x[1]),re.sub("[Ç]","C",x[2]),re.sub("[Ç]","C",x[3])])
name_df["ALL_PARTY_NAMES"]=name_df["ALL_PARTY_NAMES"].apply(lambda x: [re.sub("[Ñ]","N",x[0]),re.sub("[Ñ]","N",x[1]),re.sub("[Ñ]","N",x[2]),re.sub("[Ñ]","N",x[3])])
name_df["ALL_PARTY_NAMES"]=name_df["ALL_PARTY_NAMES"].apply(lambda x: [re.sub("[Ý]","Y",x[0]),re.sub("[Ý]","Y",x[1]),re.sub("[Ý]","Y",x[2]),re.sub("[Ý]","Y",x[3])])
                                                                                              
    
name_df["ALL_PARTY_NAMES"]=name_df["ALL_PARTY_NAMES"].apply(lambda x: [i.replace(r"\xa0","") for i in x ]) 


In [193]:
name_df["ALL_PARTY_NAMES"]=name_df["ALL_PARTY_NAMES"].apply(lambda x: [i.replace(r"\xa0","") for i in x ])

In [194]:
print(df[df["V_REGD_PARTY_NAME"].str.contains(r"\xa0")]["V_REGD_PARTY_NAME"])

Series([], Name: V_REGD_PARTY_NAME, dtype: object)


### Punctuation Marks

In [195]:
print(df[df["V_REGD_PARTY_NAME"].str.contains(", ")]["V_REGD_PARTY_NAME"])
df[df["V_REGD_PARTY_NAME"].str.contains(",\s")]["V_REGD_PARTY_NAME"]
df[df["V_REGD_PARTY_NAME"].str.contains("\s+,\s+")]["V_REGD_PARTY_NAME"]

<>:2: DeprecationWarning: invalid escape sequence \s
<>:3: DeprecationWarning: invalid escape sequence \s
<>:2: DeprecationWarning: invalid escape sequence \s
<>:3: DeprecationWarning: invalid escape sequence \s
<ipython-input-195-5519aab0cf22>:2: DeprecationWarning: invalid escape sequence \s
  df[df["V_REGD_PARTY_NAME"].str.contains(",\s")]["V_REGD_PARTY_NAME"]
<ipython-input-195-5519aab0cf22>:3: DeprecationWarning: invalid escape sequence \s
  df[df["V_REGD_PARTY_NAME"].str.contains("\s+,\s+")]["V_REGD_PARTY_NAME"]


2403      ZHANG, HONGYING
2407          ZHAN, PUWEN
2415             LIU, JIA
2570          ZHANG, MENG
2572          ZHANG, PENG
               ...       
715208         WANG, QIAN
715214      LI, XIANGGANG
715221         ZHANG, YAN
715223        ZHANG, YING
715260       LU, YUANYUAN
Name: V_REGD_PARTY_NAME, Length: 5934, dtype: object


21368               LIN , CHIH-HUNG
26311                     WU , PENG
39719                 LI , JINXIANG
54283                 YANG , XIAOYI
61788                 ZHANG , JUNHE
75232               LIN , CHIH-HUNG
86477                WANG , DANPING
93396                   YE , WEIMIN
110366                 FU , XIAOHUI
110899                NI , YINGJIAN
118229                YANG , XIAOYI
118271                    WU , PENG
120844                    WU , PENG
126049                 DU , ZHENGUO
129658                 CHEN , YANBO
144841    HOW CHOON MING , GEOFFREY
174712             CHEN , YONGCHANG
186617          LEE KAI JIE , KEVIN
209477                   LYU , JING
222774                    MA , YONG
258431                 FU , XIAOHUI
274524                SHI , HAIMING
283932               CHEN , XIAOWEI
298047                    WU , PENG
317637                CHU , JIANGEN
329944              LIN , CHIH-HUNG
347443             CHEN , YONGCHANG
372827                YANG ,

In [196]:
# check under REGD_PARTY_NAMES for punctuations
name_df[name_df["ALL_PARTY_NAMES"].apply(lambda x: len(re.findall("\{.*\}|\[.*\]|\\\\|:|;|\"|<|>|\?",x[0]))>0)]["ALL_PARTY_NAMES"]

<>:2: DeprecationWarning: invalid escape sequence \{
<>:2: DeprecationWarning: invalid escape sequence \{
<ipython-input-196-7f1a0ac9901a>:2: DeprecationWarning: invalid escape sequence \{
  name_df[name_df["ALL_PARTY_NAMES"].apply(lambda x: len(re.findall("\{.*\}|\[.*\]|\\\\|:|;|\"|<|>|\?",x[0]))>0)]["ALL_PARTY_NAMES"]


Series([], Name: ALL_PARTY_NAMES, dtype: object)

In [197]:
name_df["ALL_PARTY_NAMES"]

0         [SAHARUDDIN BIN MD SALLEH, , , ]
1                    [HO KIONG CHAN, , , ]
2           [AHMAD ASABI BIN ISMAIL, , , ]
3                      [ANG LEE ENG, , , ]
4                      [ENG LEE ANG, , , ]
                        ...               
715256               [MUHAMMAD UMAR, , , ]
715257               [MUHAMMAD UMAR, , , ]
715258              [MUHAMMAD UMAIR, , , ]
715259                [MAIKO SUZUKI, , , ]
715260                [LUA YUANYUAN, , , ]
Name: ALL_PARTY_NAMES, Length: 715261, dtype: object

In [198]:
#comma
name_df["ALL_PARTY_NAMES"]=name_df["ALL_PARTY_NAMES"].apply(lambda x: [re.sub("\s+,\s+"," ",i) for i in x])
name_df["ALL_PARTY_NAMES"]=name_df["ALL_PARTY_NAMES"].apply(lambda x: [re.sub("\s+,"," ",i) for i in x])
name_df["ALL_PARTY_NAMES"]=name_df["ALL_PARTY_NAMES"].apply(lambda x: [re.sub(",\s+"," ",i) for i in x])
name_df["ALL_PARTY_NAMES"]=name_df["ALL_PARTY_NAMES"].apply(lambda x: [re.sub(","," ",i) for i in x])

<>:1: DeprecationWarning: invalid escape sequence \s
<>:2: DeprecationWarning: invalid escape sequence \s
<>:3: DeprecationWarning: invalid escape sequence \s
<>:1: DeprecationWarning: invalid escape sequence \s
<>:2: DeprecationWarning: invalid escape sequence \s
<>:3: DeprecationWarning: invalid escape sequence \s
<ipython-input-198-01fe266f7ac3>:1: DeprecationWarning: invalid escape sequence \s
  name_df["ALL_PARTY_NAMES"]=name_df["ALL_PARTY_NAMES"].apply(lambda x: [re.sub("\s+,\s+"," ",i) for i in x])
<ipython-input-198-01fe266f7ac3>:2: DeprecationWarning: invalid escape sequence \s
  name_df["ALL_PARTY_NAMES"]=name_df["ALL_PARTY_NAMES"].apply(lambda x: [re.sub("\s+,"," ",i) for i in x])
<ipython-input-198-01fe266f7ac3>:3: DeprecationWarning: invalid escape sequence \s
  name_df["ALL_PARTY_NAMES"]=name_df["ALL_PARTY_NAMES"].apply(lambda x: [re.sub(",\s+"," ",i) for i in x])


In [199]:
# We have need to use \\ if we need to check \ in text.We see no \ slash is there in names
new_df["V_REGD_PARTY_NAME"]=new_df["V_REGD_PARTY_NAME"].apply (lambda x: "" if pd.isna(x) else x)
new_df[new_df["V_REGD_PARTY_NAME"].str.contains(" \\ ")]["V_REGD_PARTY_NAME"]
new_df["V_PREVIOUS_NAME_1"]=new_df["V_PREVIOUS_NAME_1"].apply (lambda x: "" if pd.isna(x) else x)
new_df[new_df["V_PREVIOUS_NAME_1"].str.contains(" \\ ")]["V_PREVIOUS_NAME_1"]
new_df["V_PREVIOUS_NAME_2"]=new_df["V_PREVIOUS_NAME_2"].apply (lambda x: "" if pd.isna(x) else x)
new_df[new_df["V_PREVIOUS_NAME_2"].str.contains(" \\ ")]["V_PREVIOUS_NAME_2"]
new_df["V_PREVIOUS_NAME_IMMEDIATE"]=new_df["V_PREVIOUS_NAME_IMMEDIATE"].apply (lambda x: "" if pd.isna(x) else x)
new_df[new_df["V_PREVIOUS_NAME_IMMEDIATE"].str.contains(" \\ ")]["V_PREVIOUS_NAME_IMMEDIATE"]


Series([], Name: V_PREVIOUS_NAME_IMMEDIATE, dtype: object)

In [200]:
scr_rules_2
for j in scr_rules_2["V_SCREENING_VALUE_1"]:
    print(j)
    name_df["ALL_PARTY_NAMES"]=name_df["ALL_PARTY_NAMES"].apply(lambda x: [re.sub(j," ",x[i]) for i in range(len(x))])

~
!
#
\$
%
\^
&
\*
_
-
\+
=
\{
\}
\[
\]
\|
\\\\
:
;
\"
<
>
\?


### Standardized Names

In [201]:
name_df["ALL_PARTY_NAMES"]

0         [SAHARUDDIN BIN MD SALLEH, , , ]
1                    [HO KIONG CHAN, , , ]
2           [AHMAD ASABI BIN ISMAIL, , , ]
3                      [ANG LEE ENG, , , ]
4                      [ENG LEE ANG, , , ]
                        ...               
715256               [MUHAMMAD UMAR, , , ]
715257               [MUHAMMAD UMAR, , , ]
715258              [MUHAMMAD UMAIR, , , ]
715259                [MAIKO SUZUKI, , , ]
715260                [LUA YUANYUAN, , , ]
Name: ALL_PARTY_NAMES, Length: 715261, dtype: object

In [202]:
scr_rules_3[["V_SCREENING_VALUE_1","Solution"]]

,V_SCREENING_VALUE_1,Solution
45,MUHAMMAD |MUHAMMED |MUHAMAD |MUHAMED |MUHD,MUHD
46,MOHAMMAD |MOHAMMED |MOHAMAD |MOHAMED | MOHD,MOHD
47,ABDUL |ABDOUL |ABDUR |ABDOL |ABDOOL |ABD |AB,ABDUL
48,NOOR |NOR |NUUR |NUR,NOOR
49,AHMAD |AHMED |AHMET,AHMAD
50,ABDULLAH |ABDOLLAH,ABDULLAH
51,YUSOFF |YUSOF |YUSUFF |YUSUF,YUSOFF


In [203]:
std_nam_ind=df[df["V_REGD_PARTY_NAME"].str.contains("MOHAMMAD")]["V_REGD_PARTY_NAME"].index
df[df["V_REGD_PARTY_NAME"].str.contains("MOHAMMAD")]["V_REGD_PARTY_NAME"]

288       MOHAMMAD AFIQ IZZUDDIN BIN ZAINOL
366                 MOHAMMAD FAIZ BIN SAKRI
401                     MOHAMMAD OMAR FARUK
403                     MOHAMMAD OMAR FARUK
414       MOHAMMAD FAZRIN HELMI BIN JA'AFAR
                        ...                
714702         MOHAMMAD NABIEL BIN SHOFHIAN
714704                MOHAMMAD ABDUR RAHMAN
714785             MOHAMMAD SYAFIK ABDULLAH
714807         MOHAMMAD SHAFIQ BIN ABDULLAH
714818         MOHAMMAD SHAFIQ BIN ABDULLAH
Name: V_REGD_PARTY_NAME, Length: 15840, dtype: object

In [204]:
for k in scr_rules_3[["V_SCREENING_VALUE_1","Solution"]].values:
    print(k[0],k[1])
    name_df["ALL_PARTY_NAMES"]=name_df["ALL_PARTY_NAMES"].apply(lambda x: [re.sub(k[0],k[1],x[i]) for i in range(len(x))])

MUHAMMAD |MUHAMMED |MUHAMAD |MUHAMED |MUHD   MUHD 
MOHAMMAD |MOHAMMED |MOHAMAD |MOHAMED | MOHD   MOHD 
ABDUL |ABDOUL |ABDUR |ABDOL |ABDOOL |ABD |AB   ABDUL 
NOOR |NOR |NUUR |NUR   NOOR 
AHMAD |AHMED |AHMET   AHMAD 
ABDULLAH |ABDOLLAH   ABDULLAH 
YUSOFF |YUSOF |YUSUFF |YUSUF   YUSOFF 


In [205]:
name_df.loc[std_nam_ind,"ALL_PARTY_NAMES"]

288       [ MOHD AFIQ IZZUDDIN BIN ZAINOL, , , ]
366                 [ MOHD FAIZ BIN SAKRI, , , ]
401                     [ MOHD OMAR FARUK, , , ]
403                     [ MOHD OMAR FARUK, , , ]
414        [ MOHD FAZRIN HELMI BIN JAAFAR, , , ]
                           ...                  
714702         [ MOHD NABIEL BIN SHOFHIAN, , , ]
714704               [ MOHD  ABDUL RAHMAN, , , ]
714785             [ MOHD SYAFIK ABDULLAH, , , ]
714807         [ MOHD SHAFIQ BIN ABDULLAH, , , ]
714818         [ MOHD SHAFIQ BIN ABDULLAH, , , ]
Name: ALL_PARTY_NAMES, Length: 15840, dtype: object

In [206]:
# For any other miscellaneous character
name_df["ALL_PARTY_NAMES"]=name_df["ALL_PARTY_NAMES"].apply(lambda x : [re.sub("[^A-Z\s ]+","",x[i]) for i in range(len(x))])

<>:2: DeprecationWarning: invalid escape sequence \s
<>:2: DeprecationWarning: invalid escape sequence \s
<ipython-input-206-454ab968eabe>:2: DeprecationWarning: invalid escape sequence \s
  name_df["ALL_PARTY_NAMES"]=name_df["ALL_PARTY_NAMES"].apply(lambda x : [re.sub("[^A-Z\s ]+","",x[i]) for i in range(len(x))])


In [207]:
name_df[name_df["V_ALIAS_NAME_3"].apply(lambda x: np.any(type(x)!=list))]
#name_df["V_ALIAS_NAME_4"]
#name_df["GROUP_ALIAS_NAME_1"].apply

,ALL_PARTY_NAMES,V_ALIAS_NAME_2,V_ALIAS_NAME_3,V_ALIAS_NAME_4


In [208]:
# V_ALIAS_NAME - Punctuation mark , std names , miscellaneous character
name_df["GROUP_ALIAS_NAME_1"]=name_df[["V_ALIAS_NAME_2","V_ALIAS_NAME_3","V_ALIAS_NAME_4"]].apply(lambda x: x[0]+x[1]+x[2],axis=1)
scr_rules_2
for i in scr_rules_2["V_SCREENING_VALUE_1"]:
    print(i)
    #name_df["V_ALIAS_NAME_2"]=name_df["V_ALIAS_NAME_2"].apply(lambda x: [re.sub(i," ",x[j]) for j in range(len(x))])
    name_df["GROUP_ALIAS_NAME_1"]=name_df["GROUP_ALIAS_NAME_1"].apply(lambda x: [re.sub(str(i)," ",x[j]) for j in range(len(x))])

#Std Names
for k in scr_rules_3[["V_SCREENING_VALUE_1","Solution"]].values:
    print(k[0],k[1])
    name_df["GROUP_ALIAS_NAME_1"]=name_df["GROUP_ALIAS_NAME_1"].apply(lambda x: [re.sub(k[0],k[1],x[i]) for i in range(len(x))])


 # Miscellaneous character
name_df["GROUP_ALIAS_NAME_1"]=name_df["GROUP_ALIAS_NAME_1"].apply(lambda x : [re.sub("[^A-Z\s ]+","",x[i]) for i in range(len(x)) ])                                                                  

<>:16: DeprecationWarning: invalid escape sequence \s
<>:16: DeprecationWarning: invalid escape sequence \s
<ipython-input-208-f0643fbffc56>:16: DeprecationWarning: invalid escape sequence \s
  name_df["GROUP_ALIAS_NAME_1"]=name_df["GROUP_ALIAS_NAME_1"].apply(lambda x : [re.sub("[^A-Z\s ]+","",x[i]) for i in range(len(x)) ])


~
!
#
\$
%
\^
&
\*
_
-
\+
=
\{
\}
\[
\]
\|
\\\\
:
;
\"
<
>
\?
MUHAMMAD |MUHAMMED |MUHAMAD |MUHAMED |MUHD   MUHD 
MOHAMMAD |MOHAMMED |MOHAMAD |MOHAMED | MOHD   MOHD 
ABDUL |ABDOUL |ABDUR |ABDOL |ABDOOL |ABD |AB   ABDUL 
NOOR |NOR |NUUR |NUR   NOOR 
AHMAD |AHMED |AHMET   AHMAD 
ABDULLAH |ABDOLLAH   ABDULLAH 
YUSOFF |YUSOF |YUSUFF |YUSUF   YUSOFF 


# Watch-list names Pre-processing

In [209]:
scr_rules

,Sequence of the rules,col_1,V_SCREENING_VALUE_1,Remarks,Solution,V_SCREENING_VALUE_2,col_6
0,1,Patronym,A/L | A/P | S/O | D/O | SO | DO | AP,Terms are used to signify the father's name like A/P refers to Daughter of. For eg. VERNISHA J...,We remove these these terms,NaN,NaN
1,1,Patronym,BINTI | BINTE | B | BTE | BT | IBNI | IBNU | ANAK,Son / daughter of. for Malay & indegenous people. For eg. FARIDAH BINTE MAHMUD,We remove these these terms,NaN,NaN
2,2,Apostrophe,\',Change in sound,Remove it,NaN,NaN
3,3,Hyphen,[-],use to join words,Replace with white space character,NaN,NaN
4,4,Prefix,HAJI |HAJJAH |HAJAH |HJH |HJ |ALHAJ |HADJI,Tilte for˜Muslim˜men and women who have performed the˜Hajj (pilgrimage). That means these terms ...,Remove the terms if present in Individual( also in father's name if it comes),NaN,NaN
5,4,Prefix,WIRA |PANGLIMA |IR |DR. |PROF. |SR,Other salutations; professional rank,Remove the salutations; professional rank,NaN,NaN
6,5,Titles for Royal and honoured people,K.D.Y.T.M. |K.D.Y.M.M. |D.Y.M.M. |D.Y.T.M. |Y.T.M. |D.Y.A.M. |Y.M.M. |Y.A.M. |Y.BHG. |KDYTM |KDY...,Abbreviation for the king title,Remove the title,NaN,NaN
7,5,Titles for Royal and honoured people,YANG DI PERTUAN AGONG |DULI YANG MAHA MULIA SERI PADUKA BAGINDA,Official title of the king(ruler) of all Malaysia,Remove the title,NaN,NaN
8,5,Titles for Royal and honoured people,TIMBALAN YANG DI PERTUAN AGONG,NaN,Remove the title,NaN,NaN
9,5,Titles for Royal and honoured people,YANG DI PERTUAN BESAR |YANG DI PERTUA NEGERI |SULTAN,NaN,Remove the title,NaN,NaN


In [210]:
df["V_PROFILE_NAME"]=new_df["V_PROFILE_NAME"].values

In [211]:
df["V_PROFILE_NAME"]=df["V_PROFILE_NAME"].apply(lambda x: "" if pd.isna(x) else x)

In [212]:
df[df["V_PROFILE_NAME"].str.contains("\'")]["V_PROFILE_NAME"]

414       MOHAMMAD FAZRIN HELMI BIN JA'AFAR
588             ABDUL MU'AZ BIN ABDUL RAZAK
589             ABDUL MU'AZ BIN ABDUL RAZAK
590             ABDUL MU'AZ BIN ABDUL RAZAK
933                   RAHIMAH BINTI YA'ACOB
                        ...                
713291             MOHD 'AZIM BIN MAT YUSOF
713292             MOHD 'AZIM BIN MAT YUSOF
713293             MOHD 'AZIM BIN MAT YUSOF
713294             MOHD 'AZIM BIN MAT YUSOF
713310                       H'NG YEAT KOOI
Name: V_PROFILE_NAME, Length: 1188, dtype: object

In [213]:
print(df[df["V_PROFILE_NAME"].str.contains(", ")]["V_PROFILE_NAME"])
df[df["V_PROFILE_NAME"].str.contains(",")]["V_PROFILE_NAME"]

34446                          LIANG, SHUANG
61154                           XIAO, LINLIN
71666                            WANG, XUHUI
94269                                LI, WEI
94270                                LI, WEI
94271                                LI, WEI
94272                                LI, WEI
102030                          CAI, LONGWEI
105659                           SUN, XINXIN
134924                        FENG, SHUANGXI
144841              HOW CHOON MING, GEOFFREY
159135                          WANG, DETONG
161365                            ZHANG, YUE
185698                         HOU, ZHANGPAN
186617                   LEE KAI JIE , KEVIN
238400                      HSIA, HSIEN-TANG
252559                               YU, LIN
274268                         ZHOU, NINGHAO
343188                            YING, KUAN
399344                            WEN, FULAN
405329                           LIU, GUOJUN
421733                           LI, HAIHONG
421734    

4334                    Tet Hau,NG
4335                    Tet Hau,NG
4336              Van Cuong,NGUYEN
4337              Van Cuong,NGUYEN
22574                Kuen Yin,WONG
                    ...           
689479    KUMARAGURU THANASEKARAN,
693520        Justin Yung Bin,WONG
694250                HU, JIANPING
698963                 LIU, ZHENYU
699288               ZHAO, JINGHUI
Name: V_PROFILE_NAME, Length: 483, dtype: object

In [214]:
# Patronym , ' - , Honourary Titles 
df["V_PROFILE_NAME"]=df["V_PROFILE_NAME"].str.upper()
for k in [' A/L | A/P | S/O | D/O | SO | DO | AP ',' BINTI | BINTE | B | BTE | BT | IBNI | IBNU | ANAK ']:  # if any patronym title comes , then remove it 
    df["V_PROFILE_NAME"]=df["V_PROFILE_NAME"].apply(lambda x: re.sub(k," ",x))
#' apostrophe Abd Alfattah So'ud Abd Alfattah DARDAS
df["V_PROFILE_NAME"]=df["V_PROFILE_NAME"].apply(lambda x: re.sub("\'","",x))
# - dash
df["V_PROFILE_NAME"]=df["V_PROFILE_NAME"].apply(lambda x: re.sub("-"," ",x))

for k in scr_rules["V_SCREENING_VALUE_1"].values[4:16].tolist():
    print(k)
    df["V_PROFILE_NAME"]=df["V_PROFILE_NAME"].apply(lambda x: re.sub("^("+k+")","",x)) 
   
# We need to remove HAJI & all such terms if in case they come in b/w names.
df["V_PROFILE_NAME"]=df["V_PROFILE_NAME"].apply(lambda x: re.sub(" HAJI | HAJJAH | HAJAH | HJH | HJ | ALHAJ | HADJI "," ",x))
df["V_PROFILE_NAME"]=df["V_PROFILE_NAME"].apply(lambda x: re.sub("( HAJI| HAJJAH| HAJAH| HJH| HJ| ALHAJ| HADJI)$","",x))


        # Need to add a cap ^ to remove only the names where titles come as prefix.


HAJI |HAJJAH |HAJAH |HJH |HJ |ALHAJ |HADJI 
WIRA |PANGLIMA |IR |DR. |PROF. |SR 
K.D.Y.T.M. |K.D.Y.M.M. |D.Y.M.M. |D.Y.T.M. |Y.T.M. |D.Y.A.M. |Y.M.M. |Y.A.M. |Y.BHG. |KDYTM |KDYMM |DYMM |DYTM |DYAM |YMM |YTM 
YANG  DI PERTUAN  AGONG |DULI  YANG  MAHA  MULIA  SERI  PADUKA  BAGINDA 
TIMBALAN  YANG  DI PERTUAN  AGONG 
YANG DI PERTUAN BESAR |YANG  DI PERTUA  NEGERI  |SULTAN 
RAJA  PERMAISURI  AGONG 
TUNKU  AMPUAN  BESAR |TUNKU  AMPUAN |TUNKU  PUAN  BESAR |SULTANAH |TENGKU  AMPUAN  BESAR |TENGKU  AMPUAN  MAHKOTA |TENGKU  AMPUAN | TENGKU  BESAR |TENGKU  MAHKOTA |PADUKA  BONDA  RAJA 
RAJA  PERMAISURI |PERMAISURI |RAJA  MUDA |RAJA  PUAN  MUDA |RAJA  PUAN  BESAR |RAJA  DI HILIR |TENGKU  PUAN  MUDA |TENGKU  PUAN |YANG  DI-PERTUAN  MUDA |YANG  DI-PERTUAN  BESAR 
TUANKU |TUNKU |TUN 
RAJA |TENGKU |MEK |NIK |TUAN |WAN |PUTERI |PUTRI |AWANG |ABANG |DAYANG |CIK  PUAN  BESAR |CIK  PUAN |ENCHE  BESAR |CHE  PUAN  BESAR |SYED |SY |SAYED |SHARIFAH |SH |SYARIFAH 
TAN  SRI |DATUK  SERI |DATUK  SRI |DATUK |DAT

In [215]:
'''
df[df["V_PROFILE_NAME"].str.contains("\(.+\)")]["V_PROFILE_NAME"]
LEE SOO HUA (LAI HUAT) & SONS SDN BHD
38919     LEE SOO HUA (LAI HUAT) & SONS SDN BHD
38920     LEE SOO HUA (LAI HUAT) & SONS SDN BHD
38921     LEE SOO HUA (LAI HUAT) & SONS SDN BHD
38922     LEE SOO HUA (LAI HUAT) & SONS SDN BHD
38923     LEE SOO HUA (LAI HUAT) & SONS SDN BHD
'''

<>:9: DeprecationWarning: invalid escape sequence \(
<>:9: DeprecationWarning: invalid escape sequence \(
<ipython-input-215-9611803cf97d>:9: DeprecationWarning: invalid escape sequence \(
  '''


'\ndf[df["V_PROFILE_NAME"].str.contains("\\(.+\\)")]["V_PROFILE_NAME"]\nLEE SOO HUA (LAI HUAT) & SONS SDN BHD\n38919     LEE SOO HUA (LAI HUAT) & SONS SDN BHD\n38920     LEE SOO HUA (LAI HUAT) & SONS SDN BHD\n38921     LEE SOO HUA (LAI HUAT) & SONS SDN BHD\n38922     LEE SOO HUA (LAI HUAT) & SONS SDN BHD\n38923     LEE SOO HUA (LAI HUAT) & SONS SDN BHD\n'

In [216]:
df.shape

(715261, 112)

In [217]:
df[df["V_PROFILE_NAME"].str.contains("\(.+\)")]["V_PROFILE_NAME"]

<>:1: DeprecationWarning: invalid escape sequence \(
<>:1: DeprecationWarning: invalid escape sequence \(
<ipython-input-217-595d13221d89>:1: DeprecationWarning: invalid escape sequence \(
  df[df["V_PROFILE_NAME"].str.contains("\(.+\)")]["V_PROFILE_NAME"]


46777                SOON (SOIN) CHIN WEI
64317        BARRY TAN CHONG LIANG (DATO)
77506                 NGE (NG) JIAN XIANG
130342               KER KIM LOONG (DATO)
170299         YEO KAY HIAN (YANG QIXIAN)
319555    MOHAMMAD REZUANUL HOQUE (JEWEL)
457490        SEAH KOK PENG (SHE GUOPING)
526406    SEOW CHOON CHOON(XIAO CHUNCHUN)
689932             OOI YEOW TATT(BERNARD)
Name: V_PROFILE_NAME, dtype: object

In [218]:
df[df["V_PROFILE_NAME"].str.contains("@")]["V_PROFILE_NAME"]

460          AMINUDDIN BIN MD SOM @ ABDULLAH
902             RAMARAO @ JANAGIRAM RAMASAMY
989               NG SEH HOW @ NG KHAI SIONG
1010          BEH TEK LOONG @ MAH PENG CHEAH
1133                 ZAID @ ZAIDI BIN KHALID
                         ...                
708735    MOHD SHULGA BIN NULGA @ MAD HARIAN
711180              NORIZAN MOHAMAD@MOHAMMAD
712097                 NAZRI BIN SANTA @ EDD
713195        MOHD NAFEE @ NAFIEE BIN KASSIM
715103     MOHD YADEY BIN IDRIS @ MOHD IDRIS
Name: V_PROFILE_NAME, Length: 1141, dtype: object

In [219]:
df["V_PROFILE_NAME"]

0         SHARUDDIN BIN MD SALLEH
1                   HO KIONG CHAN
2          AHMAD ASABI BIN ISMAIL
3                     ANG LEE ENG
4                     ANG LEE ENG
                   ...           
715256             MUHAMMAD UMAIR
715257             MUHAMMAD UMAIR
715258             MUHAMMAD UMAIR
715259              MARIKO SUZUKI
715260               YUANYUAN LIU
Name: V_PROFILE_NAME, Length: 715261, dtype: object

In [220]:
# V_WATCHLIST_ALIAS_NAME
# using () - assuming that ( comes only for once)
df["V_WATCHLIST_ALIAS_NAME_2"]=df["V_PROFILE_NAME"].apply(lambda x: re.findall("\(.+\)",x))
df["V_WATCHLIST_ALIAS_NAME_2"]=df["V_WATCHLIST_ALIAS_NAME_2"].apply(lambda x: x[0].lstrip("(").rstrip(")") if len(x)>0 else "" )

df["V_PROFILE_NAME"]=df["V_PROFILE_NAME"].apply(lambda x: re.sub(" \(.+\)","",x))


#@ - assuming it comes only for once
df["V_WATCHLIST_ALIAS_NAME_3"]=[""]*(df.shape[0])
df["V_PROFILE_NAME"]=df["V_PROFILE_NAME"].apply(lambda x: re.split("@",x))
df["V_WATCHLIST_ALIAS_NAME_3"]=df["V_PROFILE_NAME"].apply(lambda x: x[1].lstrip(" ") if len(x) >1 else "")
df["V_PROFILE_NAME"]=df["V_PROFILE_NAME"].apply( lambda x: x[0].rstrip(" "))

# slash - assume that / can come several times & therefore we split .
df["V_WATCHLIST_ALIAS_NAME_4"]=[""]*(df.shape[0])
df["V_WATCHLIST_ALIAS_NAME_4"]=df["V_PROFILE_NAME"].str.split(" / ")
df["V_PROFILE_NAME"]=df["V_WATCHLIST_ALIAS_NAME_4"].apply(lambda x: x[0])
df["V_WATCHLIST_ALIAS_NAME_4"]=df["V_WATCHLIST_ALIAS_NAME_4"].apply(lambda x: x[1:2] if len(x)>1 else [])

# comma 
df["V_PROFILE_NAME"]=df["V_PROFILE_NAME"].apply(lambda x: re.sub("\s+,\s+"," ",x) )
df["V_PROFILE_NAME"]=df["V_PROFILE_NAME"].apply(lambda x: re.sub("\s+,"," ",x))
df["V_PROFILE_NAME"]=df["V_PROFILE_NAME"].apply(lambda x: re.sub(",\s+"," ",x) )
df["V_PROFILE_NAME"]=df["V_PROFILE_NAME"].apply(lambda x: re.sub(","," ",x) )
#df["V_WATCHLIST_ALIAS_NAME_5"]=df["V_PROFILE_NAME"].apply(lambda x: x[1].lstrip(" ") if len(x) >1 else "")
#df["V_PROFILE_NAME"]=df["V_PROFILE_NAME"].apply(lambda x: x[0].rstrip(" "))
             

<>:3: DeprecationWarning: invalid escape sequence \(
<>:6: DeprecationWarning: invalid escape sequence \(
<>:22: DeprecationWarning: invalid escape sequence \s
<>:23: DeprecationWarning: invalid escape sequence \s
<>:24: DeprecationWarning: invalid escape sequence \s
<>:3: DeprecationWarning: invalid escape sequence \(
<>:6: DeprecationWarning: invalid escape sequence \(
<>:22: DeprecationWarning: invalid escape sequence \s
<>:23: DeprecationWarning: invalid escape sequence \s
<>:24: DeprecationWarning: invalid escape sequence \s
<ipython-input-220-b663b535573a>:3: DeprecationWarning: invalid escape sequence \(
  df["V_WATCHLIST_ALIAS_NAME_2"]=df["V_PROFILE_NAME"].apply(lambda x: re.findall("\(.+\)",x))
<ipython-input-220-b663b535573a>:6: DeprecationWarning: invalid escape sequence \(
  df["V_PROFILE_NAME"]=df["V_PROFILE_NAME"].apply(lambda x: re.sub(" \(.+\)","",x))
<ipython-input-220-b663b535573a>:22: DeprecationWarning: invalid escape sequence \s
  df["V_PROFILE_NAME"]=df["V_PROFILE

In [221]:
df["V_PROFILE_NAME"]

0         SHARUDDIN BIN MD SALLEH
1                   HO KIONG CHAN
2          AHMAD ASABI BIN ISMAIL
3                     ANG LEE ENG
4                     ANG LEE ENG
                   ...           
715256             MUHAMMAD UMAIR
715257             MUHAMMAD UMAIR
715258             MUHAMMAD UMAIR
715259              MARIKO SUZUKI
715260               YUANYUAN LIU
Name: V_PROFILE_NAME, Length: 715261, dtype: object

In [222]:
df[df["V_PROFILE_NAME"].str.contains("\xa0")]["V_PROFILE_NAME"]

64829        MOHD NOOR AFFENDI BIN ZAUKLFLI 
105835                         TANVEER AHMED
179945           ABDUL RAZAK BIN ABDUL MANAF
205072                       YAP SOON THIAM 
222951              VALENTINE ANAK NGERANTAR
223982           SITI NORBALQIS BINTI ZAINOL
228214                       KRISHNA SELVAM 
228215                       KRISHNA SELVAM 
228216                       KRISHNA SELVAM 
228217                       KRISHNA SELVAM 
229793                          RIZWAN NAZIR
365394                   HASHILAWATI HUSSIN 
365395                   HASHILAWATI HUSSIN 
482362                        TENG CHEE KONG
482363                        TENG CHEE KONG
482364                        TENG CHEE KONG
482365                        TENG CHEE KONG
488674            MOHAMAD WAZIR BIN BAHARIN 
526209       NOOR ASIKIN BINTI MOHAMAD YUSOP
534636                ATIKAH BINTI  ABDULLAH
534637                ATIKAH BINTI  ABDULLAH
534638                ATIKAH BINTI  ABDULLAH
537406    

In [223]:
# Accented characters
df["V_PROFILE_NAME"]=df["V_PROFILE_NAME"].apply(lambda x: re.sub("[Á,Ä,Â,Ã]","A",x))
df["V_PROFILE_NAME"]=df["V_PROFILE_NAME"].apply(lambda x: re.sub("[Å]","AA",x))
df["V_PROFILE_NAME"]=df["V_PROFILE_NAME"].apply(lambda x: re.sub("[É,Ë,Ê,È]","E",x))
df["V_PROFILE_NAME"]=df["V_PROFILE_NAME"].apply(lambda x: re.sub("[Í,Ï,Î]","I",x))
df["V_PROFILE_NAME"]=df["V_PROFILE_NAME"].apply(lambda x: re.sub("[Ó,Ô,Ö,Õ]","O",x))
df["V_PROFILE_NAME"]=df["V_PROFILE_NAME"].apply(lambda x: re.sub("[Ú,Ü,Ù]","U",x))

df["V_PROFILE_NAME"]=df["V_PROFILE_NAME"].apply(lambda x: re.sub("[Ø]","OE",x))
df["V_PROFILE_NAME"]=df["V_PROFILE_NAME"].apply(lambda x: re.sub("[Ç]","C",x))
df["V_PROFILE_NAME"]=df["V_PROFILE_NAME"].apply(lambda x: re.sub("[Ñ]","N",x))
df["V_PROFILE_NAME"]=df["V_PROFILE_NAME"].apply(lambda x: re.sub("[Ý]","Y",x))
                                                                                 

In [224]:
print(df[df["V_PROFILE_NAME"].str.contains(r"\xa0")]["V_PROFILE_NAME"])


64829        MOHD NOOR AFFENDI BIN ZAUKLFLI 
105835                         TANVEER AHMED
179945           ABDUL RAZAK BIN ABDUL MANAF
205072                       YAP SOON THIAM 
222951              VALENTINE ANAK NGERANTAR
223982           SITI NORBALQIS BINTI ZAINOL
228214                       KRISHNA SELVAM 
228215                       KRISHNA SELVAM 
228216                       KRISHNA SELVAM 
228217                       KRISHNA SELVAM 
229793                          RIZWAN NAZIR
365394                   HASHILAWATI HUSSIN 
365395                   HASHILAWATI HUSSIN 
482362                        TENG CHEE KONG
482363                        TENG CHEE KONG
482364                        TENG CHEE KONG
482365                        TENG CHEE KONG
488674            MOHAMAD WAZIR BIN BAHARIN 
526209       NOOR ASIKIN BINTI MOHAMAD YUSOP
534636                ATIKAH BINTI  ABDULLAH
534637                ATIKAH BINTI  ABDULLAH
534638                ATIKAH BINTI  ABDULLAH
537406    

In [225]:
df["V_PROFILE_NAME"]=df["V_PROFILE_NAME"].str.replace(r"\xa0","")
df.loc[228563,"V_PROFILE_NAME"]

'SAZALI IBRAHIM'

In [226]:
# Punctuation Marks

In [227]:
df[df["V_PROFILE_NAME"].str.contains('\\\\')]["V_PROFILE_NAME"]

Series([], Name: V_PROFILE_NAME, dtype: object)

In [228]:
scr_rules_2

,Sequence of the rules,col_1,V_SCREENING_VALUE_1,Remarks,Solution,V_SCREENING_VALUE_2,col_6
21,10,Punctuation Marks,~,NaN,Removal + replace with space,NaN,NaN
22,10,Punctuation Marks,!,NaN,Removal + replace with space,NaN,NaN
23,10,Punctuation Marks,#,NaN,Removal + replace with space,NaN,NaN
24,10,Punctuation Marks,\$,Metacharacter,Removal + replace with space,NaN,NaN
25,10,Punctuation Marks,%,NaN,Removal + replace with space,NaN,NaN
26,10,Punctuation Marks,\^,Metacharacter,Removal + replace with space,NaN,NaN
27,10,Punctuation Marks,&,NaN,Removal + replace with space,NaN,NaN
28,10,Punctuation Marks,\*,Metacharacter,Removal + replace with space,NaN,NaN
29,10,Punctuation Marks,_,NaN,Removal + replace with space,NaN,NaN
30,10,Punctuation Marks,-,NaN,Removal + replace with space,NaN,NaN


In [229]:
df[df["V_PROFILE_NAME"].apply(lambda x: len(re.findall("\{.*\}|\[.*\]|\\\\|:|;|\"|<|>|\?",x))>0)]["V_PROFILE_NAME"]

<>:1: DeprecationWarning: invalid escape sequence \{
<>:1: DeprecationWarning: invalid escape sequence \{
<ipython-input-229-859a414c9291>:1: DeprecationWarning: invalid escape sequence \{
  df[df["V_PROFILE_NAME"].apply(lambda x: len(re.findall("\{.*\}|\[.*\]|\\\\|:|;|\"|<|>|\?",x))>0)]["V_PROFILE_NAME"]


2116                SINGH;RAMPAL
2117                SINGH;RAMPAL
2118                SINGH;RAMPAL
2119      ABDUL RAHIM;KHAIRUDDIN
2120      ABDUL RAHIM;KHAIRUDDIN
                   ...          
700453                CHENG;HONG
700454                  ZHOU;JIE
700455                  ZHOU;JIE
700456                  ZHOU;JIE
709140                  CHEN;YAN
Name: V_PROFILE_NAME, Length: 2000, dtype: object

In [230]:
#Punctuation Marks
df[df["V_PROFILE_NAME"].apply(lambda x: len(re.findall("\{.*\}|\[.*\]|\\\\|:|;|\"|<|>|\?",x))>0)]["V_PROFILE_NAME"]

<>:2: DeprecationWarning: invalid escape sequence \{
<>:2: DeprecationWarning: invalid escape sequence \{
<ipython-input-230-ac6da8fcc5aa>:2: DeprecationWarning: invalid escape sequence \{
  df[df["V_PROFILE_NAME"].apply(lambda x: len(re.findall("\{.*\}|\[.*\]|\\\\|:|;|\"|<|>|\?",x))>0)]["V_PROFILE_NAME"]


2116                SINGH;RAMPAL
2117                SINGH;RAMPAL
2118                SINGH;RAMPAL
2119      ABDUL RAHIM;KHAIRUDDIN
2120      ABDUL RAHIM;KHAIRUDDIN
                   ...          
700453                CHENG;HONG
700454                  ZHOU;JIE
700455                  ZHOU;JIE
700456                  ZHOU;JIE
709140                  CHEN;YAN
Name: V_PROFILE_NAME, Length: 2000, dtype: object

In [231]:
#Punctuation Marks
scr_rules_2
for i in scr_rules_2["V_SCREENING_VALUE_1"]:
    print(i)
    df["V_PROFILE_NAME"]=df["V_PROFILE_NAME"].apply(lambda x: re.sub(i," ",x))

~
!
#
\$
%
\^
&
\*
_
-
\+
=
\{
\}
\[
\]
\|
\\\\
:
;
\"
<
>
\?


In [232]:
df["V_PROFILE_NAME"].loc[[2118,2119]]

2118              SINGH RAMPAL
2119    ABDUL RAHIM KHAIRUDDIN
Name: V_PROFILE_NAME, dtype: object

In [233]:
# Standardized Names
for k in scr_rules_3[["V_SCREENING_VALUE_1","Solution"]].values:
    print(k[0],k[1])
    df["V_PROFILE_NAME"]=df["V_PROFILE_NAME"].apply(lambda x: re.sub(k[0],k[1],x))

MUHAMMAD |MUHAMMED |MUHAMAD |MUHAMED |MUHD   MUHD 
MOHAMMAD |MOHAMMED |MOHAMAD |MOHAMED | MOHD   MOHD 
ABDUL |ABDOUL |ABDUR |ABDOL |ABDOOL |ABD |AB   ABDUL 
NOOR |NOR |NUUR |NUR   NOOR 
AHMAD |AHMED |AHMET   AHMAD 
ABDULLAH |ABDOLLAH   ABDULLAH 
YUSOFF |YUSOF |YUSUFF |YUSUF   YUSOFF 


In [234]:
# For any other miscellaneous character
df["V_PROFILE_NAME"]=df["V_PROFILE_NAME"].apply(lambda x : re.sub("[^A-Z\s ]+","",x))

<>:2: DeprecationWarning: invalid escape sequence \s
<>:2: DeprecationWarning: invalid escape sequence \s
<ipython-input-234-bf6704fc9ef5>:2: DeprecationWarning: invalid escape sequence \s
  df["V_PROFILE_NAME"]=df["V_PROFILE_NAME"].apply(lambda x : re.sub("[^A-Z\s ]+","",x))


In [235]:
#df[df["ALL_WL_NAMES_FROM_PROFILE"].apply(lambda x: sum([1 if type(i)!=str else 0 for i in x])>0)]["ALL_WL_NAMES_FROM_PROFILE"]

In [236]:
# V_WATCHLIST_ALIAS_NAME - Punctuation mark , std names , miscellaneous character
df["ALL_WL_NAMES_FROM_PROFILE"]=df[["V_WATCHLIST_ALIAS_NAME_2","V_WATCHLIST_ALIAS_NAME_3","V_WATCHLIST_ALIAS_NAME_4"]].apply(lambda x: [x[0]]+[x[1]]+x[2],axis=1)
for i in scr_rules_2["V_SCREENING_VALUE_1"]:
    print(i)
    df["ALL_WL_NAMES_FROM_PROFILE"]=df["ALL_WL_NAMES_FROM_PROFILE"].apply(lambda x: [re.sub(i," ",x[j]) for j in range(len(x))])

#Std Names
for k in scr_rules_3[["V_SCREENING_VALUE_1","Solution"]].values:
    print(k[0],k[1])
    df["ALL_WL_NAMES_FROM_PROFILE"]=df["ALL_WL_NAMES_FROM_PROFILE"].apply(lambda x: [re.sub(k[0],k[1],x[i]) for i in range(len(x))])


 # Miscellaneous character
df["ALL_WL_NAMES_FROM_PROFILE"]=df["ALL_WL_NAMES_FROM_PROFILE"].apply(lambda x : [re.sub("[^A-Z\s ]+","",x[i]) for i in range(len(x)) ])                                                               




<>:14: DeprecationWarning: invalid escape sequence \s
<>:14: DeprecationWarning: invalid escape sequence \s
<ipython-input-236-43053ae5e917>:14: DeprecationWarning: invalid escape sequence \s
  df["ALL_WL_NAMES_FROM_PROFILE"]=df["ALL_WL_NAMES_FROM_PROFILE"].apply(lambda x : [re.sub("[^A-Z\s ]+","",x[i]) for i in range(len(x)) ])


~
!
#
\$
%
\^
&
\*
_
-
\+
=
\{
\}
\[
\]
\|
\\\\
:
;
\"
<
>
\?
MUHAMMAD |MUHAMMED |MUHAMAD |MUHAMED |MUHD   MUHD 
MOHAMMAD |MOHAMMED |MOHAMAD |MOHAMED | MOHD   MOHD 
ABDUL |ABDOUL |ABDUR |ABDOL |ABDOOL |ABD |AB   ABDUL 
NOOR |NOR |NUUR |NUR   NOOR 
AHMAD |AHMED |AHMET   AHMAD 
ABDULLAH |ABDOLLAH   ABDULLAH 
YUSOFF |YUSOF |YUSUFF |YUSUF   YUSOFF 


In [237]:
df["V_PROFILE_NAME"]

0         SHARUDDIN BIN MD SALLEH
1                   HO KIONG CHAN
2          AHMAD ASABI BIN ISMAIL
3                     ANG LEE ENG
4                     ANG LEE ENG
                   ...           
715256                 MUHD UMAIR
715257                 MUHD UMAIR
715258                 MUHD UMAIR
715259              MARIKO SUZUKI
715260               YUANYUAN LIU
Name: V_PROFILE_NAME, Length: 715261, dtype: object

In [238]:
df["ALL_WL_NAMES_FROM_PROFILE"]=df[["ALL_WL_NAMES_FROM_PROFILE","V_PROFILE_NAME"]].apply(lambda x: x[0]+[x[1]],axis=1)

In [239]:
df["ALL_WL_NAMES_FROM_PROFILE"]

0         [, , SHARUDDIN BIN MD SALLEH]
1                   [, , HO KIONG CHAN]
2         [, ,  AHMAD ASABI BIN ISMAIL]
3                     [, , ANG LEE ENG]
4                     [, , ANG LEE ENG]
                      ...              
715256                [, ,  MUHD UMAIR]
715257                [, ,  MUHD UMAIR]
715258                [, ,  MUHD UMAIR]
715259              [, , MARIKO SUZUKI]
715260               [, , YUANYUAN LIU]
Name: ALL_WL_NAMES_FROM_PROFILE, Length: 715261, dtype: object

# WL Alias Name

In [240]:
df["V_ALIASES"]=new_df["V_ALIASES"].values

In [241]:
# Alias name
df["V_ALIASES"]=df["V_ALIASES"].apply(lambda x: "" if pd.isna(x) else str(x))
df["V_ALIASES"]=df["V_ALIASES"].str.upper()

df[df["V_ALIASES"].str.contains(";")]["V_ALIASES"]


1678                                                                                   FANG,YI WEI;FANG,YIWEI
1702                                                                  CAI,JIN TIAN;CAI,JINTIAN;TSAI,CHIN-TIEN
2048                                                                                  LIU,DONG;LIU,SHENG DONG
2119                          ABD RAHIM,KHAIRUDDIN;RAHIM,KHAIRUDDIN;RAHIM,KHAIRUDDIN A;RAHIM,KHAIRUDDIN BIN A
2120                          ABD RAHIM,KHAIRUDDIN;RAHIM,KHAIRUDDIN;RAHIM,KHAIRUDDIN A;RAHIM,KHAIRUDDIN BIN A
                                                         ...                                                 
715094    ABU AKRAM;AHMED MOHAMED BAHAR;BAHAR,AHMAD;BAHAR,AHMAD MOHAMMAD ATTIEH;BAHAR,AHMED;BAHAR,AHMED MO...
715107                                                                             CHEN,ZHU QUAN;CHEN,ZHUQUAN
715243                                                                               LIN,MING-HUI;LIN,MINGHUI
715244    

In [242]:
# need to split several names
df["V_ALIASES"]=df["V_ALIASES"].str.split(";")

In [243]:
# removal of comma with white space character
#df["V_ALIASES"]=df["V_ALIASES"].apply (lambda x: [re.sub(","," ",x[i]) for i in range(len(x))])
df["V_ALIASES"]=df["V_ALIASES"].apply(lambda x: [re.sub("\s+,\s+"," ",i) for i in x])
df["V_ALIASES"]=df["V_ALIASES"].apply(lambda x: [re.sub("\s+,"," ",i) for i in x])
df["V_ALIASES"]=df["V_ALIASES"].apply(lambda x: [re.sub(",\s+"," ",i) for i in x])
df["V_ALIASES"]=df["V_ALIASES"].apply(lambda x: [re.sub(","," ",i) for i in x])

<>:3: DeprecationWarning: invalid escape sequence \s
<>:4: DeprecationWarning: invalid escape sequence \s
<>:5: DeprecationWarning: invalid escape sequence \s
<>:3: DeprecationWarning: invalid escape sequence \s
<>:4: DeprecationWarning: invalid escape sequence \s
<>:5: DeprecationWarning: invalid escape sequence \s
<ipython-input-243-8514a9650470>:3: DeprecationWarning: invalid escape sequence \s
  df["V_ALIASES"]=df["V_ALIASES"].apply(lambda x: [re.sub("\s+,\s+"," ",i) for i in x])
<ipython-input-243-8514a9650470>:4: DeprecationWarning: invalid escape sequence \s
  df["V_ALIASES"]=df["V_ALIASES"].apply(lambda x: [re.sub("\s+,"," ",i) for i in x])
<ipython-input-243-8514a9650470>:5: DeprecationWarning: invalid escape sequence \s
  df["V_ALIASES"]=df["V_ALIASES"].apply(lambda x: [re.sub(",\s+"," ",i) for i in x])


In [244]:
df["V_ALIASES"].apply(lambda x: len(x))

0         1
1         1
2         1
3         1
4         1
         ..
715256    1
715257    1
715258    1
715259    1
715260    1
Name: V_ALIASES, Length: 715261, dtype: int64

In [245]:
new_df["V_ALIASES"]=new_df["V_ALIASES"].apply(lambda x: "" if pd.isna(x) else str(x))
new_df[new_df["V_ALIASES"].str.contains(";")]["V_ALIASES"]


1678                                                                                   FANG,Yi Wei;FANG,Yiwei
1702                                                                  CAI,Jin Tian;CAI,Jintian;TSAI,Chin-Tien
2048                                                                                  LIU,Dong;LIU,Sheng Dong
2119                          ABD RAHIM,Khairuddin;RAHIM,Khairuddin;RAHIM,Khairuddin A;RAHIM,Khairuddin Bin A
2120                          ABD RAHIM,Khairuddin;RAHIM,Khairuddin;RAHIM,Khairuddin A;RAHIM,Khairuddin Bin A
                                                         ...                                                 
715094    Abu Akram;Ahmed Mohamed Bahar;BAHAR,Ahmad;BAHAR,Ahmad Mohammad Attieh;BAHAR,Ahmed;BAHAR,Ahmed Mo...
715107                                                                             CHEN,Zhu Quan;CHEN,Zhuquan
715243                                                                               LIN,Ming-Hui;LIN,Minghui
715244    

In [246]:
new_df[new_df["V_ALIASES"].str.contains("\'")]["V_ALIASES"]

7930      AKBARNEJAD,Esmaeil Ghaani;Esma'il Gha'ani;Esma'il Qa'ani;Esmail Ghaani;Esmail Ghani;Esmail Qaani...
7931      AKBARNEJAD,Esmaeil Ghaani;Esma'il Gha'ani;Esma'il Qa'ani;Esmail Ghaani;Esmail Ghani;Esmail Qaani...
7932      AKBARNEJAD,Esmaeil Ghaani;Esma'il Gha'ani;Esma'il Qa'ani;Esmail Ghaani;Esmail Ghani;Esmail Qaani...
7933      AKBARNEJAD,Esmaeil Ghaani;Esma'il Gha'ani;Esma'il Qa'ani;Esmail Ghaani;Esmail Ghani;Esmail Qaani...
7934      AKBARNEJAD,Esmaeil Ghaani;Esma'il Gha'ani;Esma'il Qa'ani;Esmail Ghaani;Esmail Ghani;Esmail Qaani...
                                                         ...                                                 
713458                                                                              HUANG,Li Ang;HUANG,Li'ang
713459                                                                              HUANG,Li Ang;HUANG,Li'ang
713460                                                                              HUANG,Li Ang;HUANG,Li'ang
713461    

In [247]:
 df[df["V_ALIASES"].apply(lambda x: type(x)==bool)]["V_ALIASES"]

Series([], Name: V_ALIASES, dtype: object)

In [248]:
 df[df["V_ALIASES"].apply(lambda x: type(x)==float)]["V_ALIASES"]

Series([], Name: V_ALIASES, dtype: object)

In [249]:
# Patronym , ' - , Honourary Titles 
for k in [' A/L | A/P | S/O | D/O | SO | DO | AP ',' BINTI | BINTE | B | BTE | BT | IBNI | IBNU | ANAK ']:  # if any patronym title comes , then remove it 
    df["V_ALIASES"]=df["V_ALIASES"].apply(lambda x: [re.sub(k," ",x[i]) for i in range(len(x))])
#' apostrophe Abd Alfattah So'ud Abd Alfattah DARDAS
df["V_ALIASES"]=df["V_ALIASES"].apply(lambda x: [re.sub("\'","",x[i]) for i in range(len(x))])
# - dash
df["V_ALIASES"]=df["V_ALIASES"].apply(lambda x: [re.sub("-"," ",x[i]) for i in range(len(x))])

for k in scr_rules["V_SCREENING_VALUE_1"].values[4:16].tolist():
    print(k)
    df["V_ALIASES"]=df["V_ALIASES"].apply(lambda x: [re.sub("^("+k+")","",x[i]) for i in range(len(x))]) 
   
# We need to remove HAJI & all such terms if in case they come in b/w names.
df["V_ALIASES"]=df["V_ALIASES"].apply(lambda x: [re.sub(" HAJI | HAJJAH | HAJAH | HJH | HJ | ALHAJ | HADJI "," ",x[i]) for i in range(len(x))])
df["V_ALIASES"]=df["V_ALIASES"].apply(lambda x: [re.sub("( HAJI| HAJJAH| HAJAH| HJH| HJ| ALHAJ| HADJI)$","",x[i]) for i in range(len(x))])


        # Need to add a cap ^ to remove only the names where titles come as prefix.


HAJI |HAJJAH |HAJAH |HJH |HJ |ALHAJ |HADJI 
WIRA |PANGLIMA |IR |DR. |PROF. |SR 
K.D.Y.T.M. |K.D.Y.M.M. |D.Y.M.M. |D.Y.T.M. |Y.T.M. |D.Y.A.M. |Y.M.M. |Y.A.M. |Y.BHG. |KDYTM |KDYMM |DYMM |DYTM |DYAM |YMM |YTM 
YANG  DI PERTUAN  AGONG |DULI  YANG  MAHA  MULIA  SERI  PADUKA  BAGINDA 
TIMBALAN  YANG  DI PERTUAN  AGONG 
YANG DI PERTUAN BESAR |YANG  DI PERTUA  NEGERI  |SULTAN 
RAJA  PERMAISURI  AGONG 
TUNKU  AMPUAN  BESAR |TUNKU  AMPUAN |TUNKU  PUAN  BESAR |SULTANAH |TENGKU  AMPUAN  BESAR |TENGKU  AMPUAN  MAHKOTA |TENGKU  AMPUAN | TENGKU  BESAR |TENGKU  MAHKOTA |PADUKA  BONDA  RAJA 
RAJA  PERMAISURI |PERMAISURI |RAJA  MUDA |RAJA  PUAN  MUDA |RAJA  PUAN  BESAR |RAJA  DI HILIR |TENGKU  PUAN  MUDA |TENGKU  PUAN |YANG  DI-PERTUAN  MUDA |YANG  DI-PERTUAN  BESAR 
TUANKU |TUNKU |TUN 
RAJA |TENGKU |MEK |NIK |TUAN |WAN |PUTERI |PUTRI |AWANG |ABANG |DAYANG |CIK  PUAN  BESAR |CIK  PUAN |ENCHE  BESAR |CHE  PUAN  BESAR |SYED |SY |SAYED |SHARIFAH |SH |SYARIFAH 
TAN  SRI |DATUK  SERI |DATUK  SRI |DATUK |DAT

In [250]:
df["V_ALIASES"].loc[1877]
df["V_ALIASES"].loc[1902]

['MAUNG MAUNG GYI']

In [251]:
new_df["V_ALIASES"].isnull().sum()/df.shape[0]

0.0

In [252]:
new_df["V_ALIASES"]=new_df["V_ALIASES"].apply(lambda x: "" if pd.isna(x) else x )
new_df[new_df["V_ALIASES"].str.contains("\(.+\)")]["V_ALIASES"]

<>:2: DeprecationWarning: invalid escape sequence \(
<>:2: DeprecationWarning: invalid escape sequence \(
<ipython-input-252-c996e266f3fa>:2: DeprecationWarning: invalid escape sequence \(
  new_df[new_df["V_ALIASES"].str.contains("\(.+\)")]["V_ALIASES"]


3416      AUNG,Nay Shwe Thway;NAY SHWE THAWAY AUNG;NAY SHWE THWAY AUNG;PHO LA PYAE (FULL MOON);PHOE LA PYA...
69442     Brigadgeneral i revolutionsgardet Mostafa Mohammad NAJJAR;Brigadier Generale Mostafa Mohammad NA...
100316    Brigadgeneral i revolutionsgardet Mostafa Mohammad NAJJAR;Brigadier Generale Mostafa Mohammad NA...
169168    Ahmad VAHIDI;Ahmad VAHIDI dandártábornok,Iszlám Forradalmi Gárda;Ahmad VAHIDI, Brigadegeneral im...
169169    Ahmad VAHIDI;Ahmad VAHIDI dandártábornok,Iszlám Forradalmi Gárda;Ahmad VAHIDI, Brigadegeneral im...
                                                         ...                                                 
669672    ABDUL MAJEED,Hafeez;ABDULMAJID,Hafiz;HAFIZ MAJID (MAJID);Hafiz Majid;MAJED,Hafez;MAJID;MAJID,Haf...
669673    ABDUL MAJEED,Hafeez;ABDULMAJID,Hafiz;HAFIZ MAJID (MAJID);Hafiz Majid;MAJED,Hafez;MAJID;MAJID,Haf...
669674    ABDUL MAJEED,Hafeez;ABDULMAJID,Hafiz;HAFIZ MAJID (MAJID);Hafiz Majid;MAJED,Hafez;MAJID;MAJID,Haf...
669675    

In [253]:
new_df[new_df["V_ALIASES"].str.contains("@")]["V_ALIASES"]

Series([], Name: V_ALIASES, dtype: object)

In [254]:
# V_WATCHLIST_ALIAS_NAME
# using ()
df["V_WATCHLIST_ALIAS_NAME_5"]=df["V_ALIASES"].apply(lambda x: [re.findall("\(.+\)",x[i]) for i in range(len(x))])
df["V_WATCHLIST_ALIAS_NAME_5"]=df["V_WATCHLIST_ALIAS_NAME_5"].apply(lambda x: [x[i][0].lstrip("(").rstrip(")") if len(x[i])>0 else ""  for i in range(len(x))])

df["V_ALIASES"]=df["V_ALIASES"].apply(lambda x: [re.sub(" \(.+\)","",x[i]) for i in range(len(x))])


#@ 
df["V_WATCHLIST_ALIAS_NAME_6"]=[""]*(df.shape[0])
df["V_ALIASES"]=df["V_ALIASES"].apply(lambda x: [re.split("@",x[i]) for i in range(len(x))])
df["V_WATCHLIST_ALIAS_NAME_6"]=df["V_ALIASES"].apply(lambda x: [x[i][1].lstrip(" ") if len(x[i]) >1 else "" for i in range(len(x))])
df["V_ALIASES"]=df["V_ALIASES"].apply( lambda x: [x[i][0].rstrip(" ") for i in range(len(x))])

# slash
df["V_WATCHLIST_ALIAS_NAME_7"]=[""]*(df.shape[0])
df["V_WATCHLIST_ALIAS_NAME_7"]=df["V_ALIASES"].apply(lambda x: [re.split(" / ",x[i]) for i in range(len(x))])
df["V_ALIASES"]=df["V_WATCHLIST_ALIAS_NAME_7"].apply(lambda x: [x[i][0]  for i in range(len(x))])
df["V_WATCHLIST_ALIAS_NAME_7"]=df["V_WATCHLIST_ALIAS_NAME_7"].apply(lambda x:[x[i][1] if len(x[i])>=2 else '' for i in range(len(x))])

'''
# comma ( create new alias name ) 
df["V_ALIASES"]=df["V_ALIASES"].apply(lambda x: re.sub(","," ",x))
#df["V_WATCHLIST_ALIAS_NAME_5"]=df["V_ALIASES"].apply(lambda x: x[1].lstrip(" ") if len(x) >1 else "")
#df["V_ALIASES"]=df["V_ALIASES"].apply(lambda x: x[0].rstrip(" "))
'''
             

<>:3: DeprecationWarning: invalid escape sequence \(
<>:6: DeprecationWarning: invalid escape sequence \(
<>:3: DeprecationWarning: invalid escape sequence \(
<>:6: DeprecationWarning: invalid escape sequence \(
<ipython-input-254-0603486583bc>:3: DeprecationWarning: invalid escape sequence \(
  df["V_WATCHLIST_ALIAS_NAME_5"]=df["V_ALIASES"].apply(lambda x: [re.findall("\(.+\)",x[i]) for i in range(len(x))])
<ipython-input-254-0603486583bc>:6: DeprecationWarning: invalid escape sequence \(
  df["V_ALIASES"]=df["V_ALIASES"].apply(lambda x: [re.sub(" \(.+\)","",x[i]) for i in range(len(x))])


'\n# comma ( create new alias name ) \ndf["V_ALIASES"]=df["V_ALIASES"].apply(lambda x: re.sub(","," ",x))\n#df["V_WATCHLIST_ALIAS_NAME_5"]=df["V_ALIASES"].apply(lambda x: x[1].lstrip(" ") if len(x) >1 else "")\n#df["V_ALIASES"]=df["V_ALIASES"].apply(lambda x: x[0].rstrip(" "))\n'

In [255]:
# Accented characters
df["V_ALIASES"]=df["V_ALIASES"].apply(lambda x: [re.sub("[Á,Ä,Â,Ã]","A",x[i]) for i in range(len(x))])
df["V_ALIASES"]=df["V_ALIASES"].apply(lambda x: [re.sub("[Å]","AA",x[i]) for i in range(len(x))])
df["V_ALIASES"]=df["V_ALIASES"].apply(lambda x: [re.sub("[É,Ë,Ê,È]","E",x[i]) for i in range(len(x))])
df["V_ALIASES"]=df["V_ALIASES"].apply(lambda x: [re.sub("[Í,Ï,Î]","I",x[i]) for i in range(len(x))])
df["V_ALIASES"]=df["V_ALIASES"].apply(lambda x: [re.sub("[Ó,Ô,Ö,Õ]","O",x[i]) for i in range(len(x))])
df["V_ALIASES"]=df["V_ALIASES"].apply(lambda x: [re.sub("[Ú,Ü,Ù]","U",x[i]) for i in range(len(x))])

df["V_ALIASES"]=df["V_ALIASES"].apply(lambda x: [re.sub("[Ø]","OE",x[i]) for i in range(len(x))])
df["V_ALIASES"]=df["V_ALIASES"].apply(lambda x: [re.sub("[Ç]","C",x[i]) for i in range(len(x))])
df["V_ALIASES"]=df["V_ALIASES"].apply(lambda x: [re.sub("[Ñ]","N",x[i]) for i in range(len(x))])
df["V_ALIASES"]=df["V_ALIASES"].apply(lambda x: [re.sub("[Ý]","Y",x[i]) for i in range(len(x))])
                                                                                 

In [256]:
df["V_ALIASES"]=df["V_ALIASES"].apply(lambda x: [i.replace(r"\xa0","") for i in x ])

In [257]:
# Punctuation Marks

In [258]:
df[df["V_ALIASES"].apply(lambda x: " ".join(x)).str.contains('\\\\')]["V_ALIASES"]

Series([], Name: V_ALIASES, dtype: object)

In [259]:
scr_rules_2

,Sequence of the rules,col_1,V_SCREENING_VALUE_1,Remarks,Solution,V_SCREENING_VALUE_2,col_6
21,10,Punctuation Marks,~,NaN,Removal + replace with space,NaN,NaN
22,10,Punctuation Marks,!,NaN,Removal + replace with space,NaN,NaN
23,10,Punctuation Marks,#,NaN,Removal + replace with space,NaN,NaN
24,10,Punctuation Marks,\$,Metacharacter,Removal + replace with space,NaN,NaN
25,10,Punctuation Marks,%,NaN,Removal + replace with space,NaN,NaN
26,10,Punctuation Marks,\^,Metacharacter,Removal + replace with space,NaN,NaN
27,10,Punctuation Marks,&,NaN,Removal + replace with space,NaN,NaN
28,10,Punctuation Marks,\*,Metacharacter,Removal + replace with space,NaN,NaN
29,10,Punctuation Marks,_,NaN,Removal + replace with space,NaN,NaN
30,10,Punctuation Marks,-,NaN,Removal + replace with space,NaN,NaN


In [260]:
df["V_ALIASES"]

0                      []
1                      []
2                      []
3                      []
4                      []
               ...       
715256                 []
715257                 []
715258                 []
715259                 []
715260    [LIUAYUAN YUAN]
Name: V_ALIASES, Length: 715261, dtype: object

In [261]:
#Punctuation Marks
df[df["V_ALIASES"].apply(lambda x: " ".join(x)).apply(lambda x: len(re.findall("\{.*\}|\[.*\]|\\\\|:|;|\"|<|>|\?",x))>0)]["V_ALIASES"]


<>:2: DeprecationWarning: invalid escape sequence \{
<>:2: DeprecationWarning: invalid escape sequence \{
<ipython-input-261-f03c68223a4a>:2: DeprecationWarning: invalid escape sequence \{
  df[df["V_ALIASES"].apply(lambda x: " ".join(x)).apply(lambda x: len(re.findall("\{.*\}|\[.*\]|\\\\|:|;|\"|<|>|\?",x))>0)]["V_ALIASES"]


69442     [BRIGADGENERAL I REVOLUTIONSGARDET MOSTAFA MOHAMMAD NAJJAR, BRIGADIER GENERALE MOSTAFA MOHAMMAD ...
100316    [BRIGADGENERAL I REVOLUTIONSGARDET MOSTAFA MOHAMMAD NAJJAR, BRIGADIER GENERALE MOSTAFA MOHAMMAD ...
169168    [AHMAD VAHIDI, AHMAD VAHIDI DANDARTABORNOKAISZLAM FORRADALMI GARDA, AHMAD VAHIDI BRIGADEGENERAL ...
169169    [AHMAD VAHIDI, AHMAD VAHIDI DANDARTABORNOKAISZLAM FORRADALMI GARDA, AHMAD VAHIDI BRIGADEGENERAL ...
169170    [AHMAD VAHIDI, AHMAD VAHIDI DANDARTABORNOKAISZLAM FORRADALMI GARDA, AHMAD VAHIDI BRIGADEGENERAL ...
169171    [AHMAD VAHIDI, AHMAD VAHIDI DANDARTABORNOKAISZLAM FORRADALMI GARDA, AHMAD VAHIDI BRIGADEGENERAL ...
169172    [AHMAD VAHIDI, AHMAD VAHIDI DANDARTABORNOKAISZLAM FORRADALMI GARDA, AHMAD VAHIDI BRIGADEGENERAL ...
169173    [AHMAD VAHIDI, AHMAD VAHIDI DANDARTABORNOKAISZLAM FORRADALMI GARDA, AHMAD VAHIDI BRIGADEGENERAL ...
169174    [AHMAD VAHIDI, AHMAD VAHIDI DANDARTABORNOKAISZLAM FORRADALMI GARDA, AHMAD VAHIDI BRIGADEGENERAL ...
169175    

In [262]:
#Punctuation Marks
scr_rules_2
for i in scr_rules_2["V_SCREENING_VALUE_1"]:
    print(i)
    df["V_ALIASES"]=df["V_ALIASES"].apply(lambda x: [re.sub(i," ",x[j]) for j in range(len(x))])

~
!
#
\$
%
\^
&
\*
_
-
\+
=
\{
\}
\[
\]
\|
\\\\
:
;
\"
<
>
\?


In [263]:
# Standardized Names
for k in scr_rules_3[["V_SCREENING_VALUE_1","Solution"]].values:
    print(k[0],k[1])
    df["V_ALIASES"]=df["V_ALIASES"].apply(lambda x: [re.sub(k[0],k[1],x[i])  for i in range(len(x))])

MUHAMMAD |MUHAMMED |MUHAMAD |MUHAMED |MUHD   MUHD 
MOHAMMAD |MOHAMMED |MOHAMAD |MOHAMED | MOHD   MOHD 
ABDUL |ABDOUL |ABDUR |ABDOL |ABDOOL |ABD |AB   ABDUL 
NOOR |NOR |NUUR |NUR   NOOR 
AHMAD |AHMED |AHMET   AHMAD 
ABDULLAH |ABDOLLAH   ABDULLAH 
YUSOFF |YUSOF |YUSUFF |YUSUF   YUSOFF 


In [264]:
# For any other miscellaneous character
df["V_ALIASES"]=df["V_ALIASES"].apply(lambda x : [re.sub("[^A-Z\s ]+","",x[i]) for i in range(len(x))])

<>:2: DeprecationWarning: invalid escape sequence \s
<>:2: DeprecationWarning: invalid escape sequence \s
<ipython-input-264-71f6a70f1a27>:2: DeprecationWarning: invalid escape sequence \s
  df["V_ALIASES"]=df["V_ALIASES"].apply(lambda x : [re.sub("[^A-Z\s ]+","",x[i]) for i in range(len(x))])


In [265]:
#all_wl_nam[all_wl_nam["ALL_WL_ALIAS_NAMES"].apply(lambda x: sum([1 if type(i)!=str else 0 for i in x])>0)]["ALL_WL_ALIAS_NAMES"]

In [266]:
# V_WATCHLIST_ALIAS_NAME - Punctuation mark , std names , miscellaneous character
all_wl_nam=pd.DataFrame({})
all_wl_nam["ALL_WL_ALIAS_NAMES"]= df[["V_WATCHLIST_ALIAS_NAME_5","V_WATCHLIST_ALIAS_NAME_6","V_WATCHLIST_ALIAS_NAME_7"]].apply(lambda x: \
                                                                                                x[0]+x[1]+x[2],axis=1 )
all_wl_nam["V_ALIASES"]=df["V_ALIASES"].values 
 
for i in scr_rules_2["V_SCREENING_VALUE_1"]:
    print(i)
    all_wl_nam["ALL_WL_ALIAS_NAMES"]=all_wl_nam["ALL_WL_ALIAS_NAMES"].apply(lambda x: [re.sub(i," ",x[j]) for j in range(len(x))])

#Std Names
for k in scr_rules_3[["V_SCREENING_VALUE_1","Solution"]].values:
    print(k[0],k[1])
    all_wl_nam["ALL_WL_ALIAS_NAMES"]=all_wl_nam["ALL_WL_ALIAS_NAMES"].apply(lambda x: [re.sub(k[0],k[1],x[i]) for i in range(len(x))])


 # Miscellaneous character
all_wl_nam["ALL_WL_ALIAS_NAMES"]=all_wl_nam["ALL_WL_ALIAS_NAMES"].apply(lambda x : [re.sub("[^A-Z\s ]+","",x[i]) for i in range(len(x)) ]   )                                                               


all_wl_nam["ALL_WL_ALIAS_NAMES"]= all_wl_nam[["V_ALIASES","ALL_WL_ALIAS_NAMES"]].apply(lambda x: \
                                                                                                            x[0]+x[1],axis=1 )
 

<>:18: DeprecationWarning: invalid escape sequence \s
<>:18: DeprecationWarning: invalid escape sequence \s
<ipython-input-266-45c0d70fc3a8>:18: DeprecationWarning: invalid escape sequence \s
  all_wl_nam["ALL_WL_ALIAS_NAMES"]=all_wl_nam["ALL_WL_ALIAS_NAMES"].apply(lambda x : [re.sub("[^A-Z\s ]+","",x[i]) for i in range(len(x)) ]   )


~
!
#
\$
%
\^
&
\*
_
-
\+
=
\{
\}
\[
\]
\|
\\\\
:
;
\"
<
>
\?
MUHAMMAD |MUHAMMED |MUHAMAD |MUHAMED |MUHD   MUHD 
MOHAMMAD |MOHAMMED |MOHAMAD |MOHAMED | MOHD   MOHD 
ABDUL |ABDOUL |ABDUR |ABDOL |ABDOOL |ABD |AB   ABDUL 
NOOR |NOR |NUUR |NUR   NOOR 
AHMAD |AHMED |AHMET   AHMAD 
ABDULLAH |ABDOLLAH   ABDULLAH 
YUSOFF |YUSOF |YUSUFF |YUSUF   YUSOFF 


In [267]:
all_wl_nam[all_wl_nam["ALL_WL_ALIAS_NAMES"].apply(lambda x: type(x)==float)]["ALL_WL_ALIAS_NAMES"]

Series([], Name: ALL_WL_ALIAS_NAMES, dtype: object)

In [268]:
df["ALL_WL_NAMES_FROM_PROFILE"].head() # contains only alias names from V_ALIASES

0    [, , SHARUDDIN BIN MD SALLEH]
1              [, , HO KIONG CHAN]
2    [, ,  AHMAD ASABI BIN ISMAIL]
3                [, , ANG LEE ENG]
4                [, , ANG LEE ENG]
Name: ALL_WL_NAMES_FROM_PROFILE, dtype: object

In [269]:
all_wl_nam["ALL_WL_NAMES_FROM_PROFILE"]=df["ALL_WL_NAMES_FROM_PROFILE"].values

In [270]:
print(all_wl_nam["ALL_WL_NAMES_FROM_PROFILE"].apply(lambda x: type(x)).unique())
print(all_wl_nam["ALL_WL_ALIAS_NAMES"].apply(lambda x: type(x)).unique())

[<class 'list'>]
[<class 'list'>]


In [271]:
all_wl_nam["ALL_WL_NAMES"]=all_wl_nam[["ALL_WL_ALIAS_NAMES","ALL_WL_NAMES_FROM_PROFILE"]].apply(lambda x: x[0]+x[1],axis=1)

In [272]:
# Final df to store all WL names from profile & Alias name
all_wl_nam["ALL_WL_NAMES"]

0           [, , , , , , SHARUDDIN BIN MD SALLEH]
1                     [, , , , , , HO KIONG CHAN]
2           [, , , , , ,  AHMAD ASABI BIN ISMAIL]
3                       [, , , , , , ANG LEE ENG]
4                       [, , , , , , ANG LEE ENG]
                           ...                   
715256                  [, , , , , ,  MUHD UMAIR]
715257                  [, , , , , ,  MUHD UMAIR]
715258                  [, , , , , ,  MUHD UMAIR]
715259                [, , , , , , MARIKO SUZUKI]
715260    [LIUAYUAN YUAN, , , , , , YUANYUAN LIU]
Name: ALL_WL_NAMES, Length: 715261, dtype: object

# Alias name 

In [273]:
# pre-processing on 'V_ALIAS_NAME' from customer
df['V_ALIAS_NAME']=df['V_ALIAS_NAME'].apply(lambda x: "" if pd.isna(x) else str(x).upper())

In [274]:
df[df["V_ALIAS_NAME"].str.contains("\'")]["V_ALIAS_NAME"]

Series([], Name: V_ALIAS_NAME, dtype: object)

In [275]:
df[df["V_ALIAS_NAME"].str.contains("-")]["V_ALIAS_NAME"]

423192    SHEIKH KHALDOON KHALIFA AL-MUBARAK
426300          SHAIKH FAHAD FAISAL AL-THANI
Name: V_ALIAS_NAME, dtype: object

In [276]:
# Patronym , ' - , Honourary Titles 
df["V_ALIAS_NAME"]=df["V_ALIAS_NAME"].str.upper()
for k in [' A/L | A/P | S/O | D/O | SO | DO | AP ',' BINTI | BINTE | B | BTE | BT | IBNI | IBNU | ANAK ']:  # if any patronym title comes , then remove it 
    df["V_ALIAS_NAME"]=df["V_ALIAS_NAME"].apply(lambda x: re.sub(k," ",x))
#' apostrophe Abd Alfattah So'ud Abd Alfattah DARDAS
df["V_ALIAS_NAME"]=df["V_ALIAS_NAME"].apply(lambda x: re.sub("\'","",x))
# - dash
df["V_ALIAS_NAME"]=df["V_ALIAS_NAME"].apply(lambda x: re.sub("-"," ",x))

for k in scr_rules["V_SCREENING_VALUE_1"].values[4:16].tolist():
    print(k)
    df["V_ALIAS_NAME"]=df["V_ALIAS_NAME"].apply(lambda x: re.sub("^("+k+")","",x)) 
   
# We need to remove HAJI & all such terms if in case they come in b/w names.
df["V_ALIAS_NAME"]=df["V_ALIAS_NAME"].apply(lambda x: re.sub(" HAJI | HAJJAH | HAJAH | HJH | HJ | ALHAJ | HADJI "," ",x))
df["V_ALIAS_NAME"]=df["V_ALIAS_NAME"].apply(lambda x: re.sub("( HAJI| HAJJAH| HAJAH| HJH| HJ| ALHAJ| HADJI)$","",x))


        # Need to add a cap ^ to remove only the names where titles come as prefix.


HAJI |HAJJAH |HAJAH |HJH |HJ |ALHAJ |HADJI 
WIRA |PANGLIMA |IR |DR. |PROF. |SR 
K.D.Y.T.M. |K.D.Y.M.M. |D.Y.M.M. |D.Y.T.M. |Y.T.M. |D.Y.A.M. |Y.M.M. |Y.A.M. |Y.BHG. |KDYTM |KDYMM |DYMM |DYTM |DYAM |YMM |YTM 
YANG  DI PERTUAN  AGONG |DULI  YANG  MAHA  MULIA  SERI  PADUKA  BAGINDA 
TIMBALAN  YANG  DI PERTUAN  AGONG 
YANG DI PERTUAN BESAR |YANG  DI PERTUA  NEGERI  |SULTAN 
RAJA  PERMAISURI  AGONG 
TUNKU  AMPUAN  BESAR |TUNKU  AMPUAN |TUNKU  PUAN  BESAR |SULTANAH |TENGKU  AMPUAN  BESAR |TENGKU  AMPUAN  MAHKOTA |TENGKU  AMPUAN | TENGKU  BESAR |TENGKU  MAHKOTA |PADUKA  BONDA  RAJA 
RAJA  PERMAISURI |PERMAISURI |RAJA  MUDA |RAJA  PUAN  MUDA |RAJA  PUAN  BESAR |RAJA  DI HILIR |TENGKU  PUAN  MUDA |TENGKU  PUAN |YANG  DI-PERTUAN  MUDA |YANG  DI-PERTUAN  BESAR 
TUANKU |TUNKU |TUN 
RAJA |TENGKU |MEK |NIK |TUAN |WAN |PUTERI |PUTRI |AWANG |ABANG |DAYANG |CIK  PUAN  BESAR |CIK  PUAN |ENCHE  BESAR |CHE  PUAN  BESAR |SYED |SY |SAYED |SHARIFAH |SH |SYARIFAH 
TAN  SRI |DATUK  SERI |DATUK  SRI |DATUK |DAT

In [277]:
'''
df[df["V_PROFILE_NAME"].str.contains("\(.+\)")]["V_PROFILE_NAME"]
LEE SOO HUA (LAI HUAT) & SONS SDN BHD
38919     LEE SOO HUA (LAI HUAT) & SONS SDN BHD
38920     LEE SOO HUA (LAI HUAT) & SONS SDN BHD
38921     LEE SOO HUA (LAI HUAT) & SONS SDN BHD
38922     LEE SOO HUA (LAI HUAT) & SONS SDN BHD
38923     LEE SOO HUA (LAI HUAT) & SONS SDN BHD
'''

<>:9: DeprecationWarning: invalid escape sequence \(
<>:9: DeprecationWarning: invalid escape sequence \(
<ipython-input-277-9611803cf97d>:9: DeprecationWarning: invalid escape sequence \(
  '''


'\ndf[df["V_PROFILE_NAME"].str.contains("\\(.+\\)")]["V_PROFILE_NAME"]\nLEE SOO HUA (LAI HUAT) & SONS SDN BHD\n38919     LEE SOO HUA (LAI HUAT) & SONS SDN BHD\n38920     LEE SOO HUA (LAI HUAT) & SONS SDN BHD\n38921     LEE SOO HUA (LAI HUAT) & SONS SDN BHD\n38922     LEE SOO HUA (LAI HUAT) & SONS SDN BHD\n38923     LEE SOO HUA (LAI HUAT) & SONS SDN BHD\n'

In [278]:
df.shape

(715261, 119)

In [279]:
df[df["V_ALIAS_NAME"].str.contains("\(.+\)")]["V_ALIAS_NAME"]

<>:1: DeprecationWarning: invalid escape sequence \(
<>:1: DeprecationWarning: invalid escape sequence \(
<ipython-input-279-47643e01810b>:1: DeprecationWarning: invalid escape sequence \(
  df[df["V_ALIAS_NAME"].str.contains("\(.+\)")]["V_ALIAS_NAME"]


Series([], Name: V_ALIAS_NAME, dtype: object)

In [280]:
df[df["V_ALIAS_NAME"].str.contains("@")]["V_ALIAS_NAME"]

558458    @ ABDUL AZIZ
Name: V_ALIAS_NAME, dtype: object

In [281]:
df[df["V_ALIAS_NAME"].str.contains("\'")]["V_ALIAS_NAME"]

Series([], Name: V_ALIAS_NAME, dtype: object)

In [282]:
df[df["V_ALIAS_NAME"].str.contains(",")]["V_ALIAS_NAME"]

186629                                 CHEE SIONG, ROBERT NG
236101    JOSE CANTERO SIENRA, JOSE ANTOLIANO CANTERO SIENRA
357935                                           HONG,GUI LU
Name: V_ALIAS_NAME, dtype: object

In [283]:
df[df["V_ALIAS_NAME"].str.contains(";")]["V_ALIAS_NAME"]

Series([], Name: V_ALIAS_NAME, dtype: object)

In [284]:
# ALIAS_NAME
# using () - considering () only comes for once
df["V_ALIAS_NAME_5"]=df["V_ALIAS_NAME"].apply(lambda x: re.findall("\(.+\)",x))
df["V_ALIAS_NAME_5"]=df["V_ALIAS_NAME_5"].apply(lambda x: x[0].lstrip("(").rstrip(")") if len(x)>0 else "" )

df["V_ALIAS_NAME"]=df["V_ALIAS_NAME"].apply(lambda x: re.sub(" \(.+\)","",x))


#@ assuming @ comes only for once
df["V_ALIAS_NAME_6"]=[""]*(df.shape[0])
df["V_ALIAS_NAME"]=df["V_ALIAS_NAME"].apply(lambda x: re.split("@",x))
df["V_ALIAS_NAME_6"]=df["V_ALIAS_NAME"].apply(lambda x: x[1].lstrip(" ") if len(x) >1 else "")
df["V_ALIAS_NAME"]=df["V_ALIAS_NAME"].apply( lambda x: x[0].rstrip(" "))

# slash - can come as many times as possible
df["V_ALIAS_NAME_7"]=[""]*(df.shape[0])
df["V_ALIAS_NAME_7"]=df["V_ALIAS_NAME"].str.split(" / ")
df["V_ALIAS_NAME"]=df["V_ALIAS_NAME_7"].apply(lambda x: x[0])
df["V_ALIAS_NAME_7"]=df["V_ALIAS_NAME_7"].apply(lambda x: x[1] if len(x)>1 else "")

# comma ( create new alias name ) 
df["V_ALIAS_NAME"]=df["V_ALIAS_NAME"].apply(lambda x: re.sub("\s+,\s+"," ",x))
df["V_ALIAS_NAME"]=df["V_ALIAS_NAME"].apply(lambda x: re.sub("\s+,"," ",x))
df["V_ALIAS_NAME"]=df["V_ALIAS_NAME"].apply(lambda x: re.sub(",\s+"," ",x))
df["V_ALIAS_NAME"]=df["V_ALIAS_NAME"].apply(lambda x: re.sub(","," ",x))

<>:3: DeprecationWarning: invalid escape sequence \(
<>:6: DeprecationWarning: invalid escape sequence \(
<>:22: DeprecationWarning: invalid escape sequence \s
<>:23: DeprecationWarning: invalid escape sequence \s
<>:24: DeprecationWarning: invalid escape sequence \s
<>:3: DeprecationWarning: invalid escape sequence \(
<>:6: DeprecationWarning: invalid escape sequence \(
<>:22: DeprecationWarning: invalid escape sequence \s
<>:23: DeprecationWarning: invalid escape sequence \s
<>:24: DeprecationWarning: invalid escape sequence \s
<ipython-input-284-2eef4135af92>:3: DeprecationWarning: invalid escape sequence \(
  df["V_ALIAS_NAME_5"]=df["V_ALIAS_NAME"].apply(lambda x: re.findall("\(.+\)",x))
<ipython-input-284-2eef4135af92>:6: DeprecationWarning: invalid escape sequence \(
  df["V_ALIAS_NAME"]=df["V_ALIAS_NAME"].apply(lambda x: re.sub(" \(.+\)","",x))
<ipython-input-284-2eef4135af92>:22: DeprecationWarning: invalid escape sequence \s
  df["V_ALIAS_NAME"]=df["V_ALIAS_NAME"].apply(lambda

In [285]:
df["V_ALIAS_NAME_7"].apply(lambda x: len(x)).unique()

array([0])

In [286]:
df["V_ALIAS_NAME"].apply(lambda x: type(x)).unique()

array([<class 'str'>], dtype=object)

In [287]:
# Accented characters
df["V_ALIAS_NAME"]=df["V_ALIAS_NAME"].apply(lambda x: re.sub("[Á,Ä,Â,Ã]","A",x))
df["V_ALIAS_NAME"]=df["V_ALIAS_NAME"].apply(lambda x: re.sub("[Å]","AA",x))
df["V_ALIAS_NAME"]=df["V_ALIAS_NAME"].apply(lambda x: re.sub("[É,Ë,Ê,È]","E",x))
df["V_ALIAS_NAME"]=df["V_ALIAS_NAME"].apply(lambda x: re.sub("[Í,Ï,Î]","I",x))
df["V_ALIAS_NAME"]=df["V_ALIAS_NAME"].apply(lambda x: re.sub("[Ó,Ô,Ö,Õ]","O",x))
df["V_ALIAS_NAME"]=df["V_ALIAS_NAME"].apply(lambda x: re.sub("[Ú,Ü,Ù]","U",x))

df["V_ALIAS_NAME"]=df["V_ALIAS_NAME"].apply(lambda x: re.sub("[Ø]","OE",x))
df["V_ALIAS_NAME"]=df["V_ALIAS_NAME"].apply(lambda x: re.sub("[Ç]","C",x))
df["V_ALIAS_NAME"]=df["V_ALIAS_NAME"].apply(lambda x: re.sub("[Ñ]","N",x))
df["V_ALIAS_NAME"]=df["V_ALIAS_NAME"].apply(lambda x: re.sub("[Ý]","Y",x))


In [288]:
df["V_ALIAS_NAME"]=df["V_ALIAS_NAME"].apply(lambda x: re.sub(r"\xa0","",x))

In [289]:
# Punctuation Marks

In [290]:
df[df["V_ALIAS_NAME"].str.contains('\\\\')]["V_ALIAS_NAME"]

Series([], Name: V_ALIAS_NAME, dtype: object)

In [291]:
scr_rules_2

,Sequence of the rules,col_1,V_SCREENING_VALUE_1,Remarks,Solution,V_SCREENING_VALUE_2,col_6
21,10,Punctuation Marks,~,NaN,Removal + replace with space,NaN,NaN
22,10,Punctuation Marks,!,NaN,Removal + replace with space,NaN,NaN
23,10,Punctuation Marks,#,NaN,Removal + replace with space,NaN,NaN
24,10,Punctuation Marks,\$,Metacharacter,Removal + replace with space,NaN,NaN
25,10,Punctuation Marks,%,NaN,Removal + replace with space,NaN,NaN
26,10,Punctuation Marks,\^,Metacharacter,Removal + replace with space,NaN,NaN
27,10,Punctuation Marks,&,NaN,Removal + replace with space,NaN,NaN
28,10,Punctuation Marks,\*,Metacharacter,Removal + replace with space,NaN,NaN
29,10,Punctuation Marks,_,NaN,Removal + replace with space,NaN,NaN
30,10,Punctuation Marks,-,NaN,Removal + replace with space,NaN,NaN


In [292]:
df[df["V_ALIAS_NAME"].apply(lambda x: len(re.findall("\{.*\}|\[.*\]|\\\\|:|;|\"|<|>|\?",x))>0)]["V_ALIAS_NAME"]

<>:1: DeprecationWarning: invalid escape sequence \{
<>:1: DeprecationWarning: invalid escape sequence \{
<ipython-input-292-b30af2ead6b7>:1: DeprecationWarning: invalid escape sequence \{
  df[df["V_ALIAS_NAME"].apply(lambda x: len(re.findall("\{.*\}|\[.*\]|\\\\|:|;|\"|<|>|\?",x))>0)]["V_ALIAS_NAME"]


Series([], Name: V_ALIAS_NAME, dtype: object)

In [293]:
#Punctuation Marks
df[df["V_ALIAS_NAME"].apply(lambda x: len(re.findall("\{.*\}|\[.*\]|\\\\|:|;|\"|<|>|\?",x))>0)]["V_ALIAS_NAME"]

<>:2: DeprecationWarning: invalid escape sequence \{
<>:2: DeprecationWarning: invalid escape sequence \{
<ipython-input-293-c4e1bef828ba>:2: DeprecationWarning: invalid escape sequence \{
  df[df["V_ALIAS_NAME"].apply(lambda x: len(re.findall("\{.*\}|\[.*\]|\\\\|:|;|\"|<|>|\?",x))>0)]["V_ALIAS_NAME"]


Series([], Name: V_ALIAS_NAME, dtype: object)

In [294]:
#Punctuation Marks
scr_rules_2
for i in scr_rules_2["V_SCREENING_VALUE_1"]:
    print(i)
    df["V_ALIAS_NAME"]=df["V_ALIAS_NAME"].apply(lambda x: re.sub(i," ",x))

~
!
#
\$
%
\^
&
\*
_
-
\+
=
\{
\}
\[
\]
\|
\\\\
:
;
\"
<
>
\?


In [295]:
df["V_ALIAS_NAME"].loc[2358]

''

In [296]:
# Standardized Names
for k in scr_rules_3[["V_SCREENING_VALUE_1","Solution"]].values:
    print(k[0],k[1])
    df["V_ALIAS_NAME"]=df["V_ALIAS_NAME"].apply(lambda x: re.sub(k[0],k[1],x))

MUHAMMAD |MUHAMMED |MUHAMAD |MUHAMED |MUHD   MUHD 
MOHAMMAD |MOHAMMED |MOHAMAD |MOHAMED | MOHD   MOHD 
ABDUL |ABDOUL |ABDUR |ABDOL |ABDOOL |ABD |AB   ABDUL 
NOOR |NOR |NUUR |NUR   NOOR 
AHMAD |AHMED |AHMET   AHMAD 
ABDULLAH |ABDOLLAH   ABDULLAH 
YUSOFF |YUSOF |YUSUFF |YUSUF   YUSOFF 


In [297]:
# For any other miscellaneous character
df["V_ALIAS_NAME"]=df["V_ALIAS_NAME"].apply(lambda x : re.sub("[^A-Z\s ]+","",x))

<>:2: DeprecationWarning: invalid escape sequence \s
<>:2: DeprecationWarning: invalid escape sequence \s
<ipython-input-297-d2fb1dea11df>:2: DeprecationWarning: invalid escape sequence \s
  df["V_ALIAS_NAME"]=df["V_ALIAS_NAME"].apply(lambda x : re.sub("[^A-Z\s ]+","",x))


In [298]:
df["V_ALIAS_NAME"]

0          
1          
2          
3          
4          
         ..
715256     
715257     
715258     
715259     
715260     
Name: V_ALIAS_NAME, Length: 715261, dtype: object

In [299]:
# V_ALIAS_NAME - Punctuation mark , std names , miscellaneous character
name_df["GROUP_ALIAS_NAME_2"]=df[["V_ALIAS_NAME_5","V_ALIAS_NAME_6","V_ALIAS_NAME_7"]].apply(lambda x: [x[0],x[1],x[2]],axis=1)
name_df["V_ALIAS_NAME"]=df["V_ALIAS_NAME"].values
scr_rules_2
for i in scr_rules_2["V_SCREENING_VALUE_1"]:
    print(i)
    name_df["GROUP_ALIAS_NAME_2"]=name_df["GROUP_ALIAS_NAME_2"].apply(lambda x: [re.sub(i," ",x[j]) for j in range(len(x))])
#Std Names
for k in scr_rules_3[["V_SCREENING_VALUE_1","Solution"]].values:
    print(k[0],k[1])
    name_df["GROUP_ALIAS_NAME_2"]=name_df["GROUP_ALIAS_NAME_2"].apply(lambda x: [re.sub(k[0],k[1],x[i]) for i in range(len(x))])


 # Miscellaneous character
name_df["GROUP_ALIAS_NAME_2"]=name_df["GROUP_ALIAS_NAME_2"].apply(lambda x : [re.sub("[^A-Z\s ]+","",x[i]) for i in range(len(x)) ]  )      
name_df["GROUP_ALIAS_NAME_2"]=name_df[["V_ALIAS_NAME","GROUP_ALIAS_NAME_2"]].apply(lambda x: [x[0]]+x[1],axis=1)

<>:15: DeprecationWarning: invalid escape sequence \s
<>:15: DeprecationWarning: invalid escape sequence \s
<ipython-input-299-91d2a20bc6b7>:15: DeprecationWarning: invalid escape sequence \s
  name_df["GROUP_ALIAS_NAME_2"]=name_df["GROUP_ALIAS_NAME_2"].apply(lambda x : [re.sub("[^A-Z\s ]+","",x[i]) for i in range(len(x)) ]  )


~
!
#
\$
%
\^
&
\*
_
-
\+
=
\{
\}
\[
\]
\|
\\\\
:
;
\"
<
>
\?
MUHAMMAD |MUHAMMED |MUHAMAD |MUHAMED |MUHD   MUHD 
MOHAMMAD |MOHAMMED |MOHAMAD |MOHAMED | MOHD   MOHD 
ABDUL |ABDOUL |ABDUR |ABDOL |ABDOOL |ABD |AB   ABDUL 
NOOR |NOR |NUUR |NUR   NOOR 
AHMAD |AHMED |AHMET   AHMAD 
ABDULLAH |ABDOLLAH   ABDULLAH 
YUSOFF |YUSOF |YUSUFF |YUSUF   YUSOFF 


In [300]:
# All customer Alias name

#name_df["GROUP_ALIAS_NAME_1"]=name_df[["V_ALIAS_NAME","V_ALIAS_NAME_5","V_ALIAS_NAME_6","V_ALIAS_NAME_7"]].apply(lambda x: [x[0],x[1],x[2],x[3],x[4]],axis=1)
name_df["ALL_ALIAS_NAMES"]=name_df[["GROUP_ALIAS_NAME_1","GROUP_ALIAS_NAME_2"]].apply(lambda x: x[0]+x[1],axis=1)

In [301]:
name_df["ALL_ALIAS_NAMES"] # Customer Alias name
name_df["ALL_PARTY_NAMES"] # Customer names
all_wl_nam["ALL_WL_NAMES"] # Watch-list names

0           [, , , , , , SHARUDDIN BIN MD SALLEH]
1                     [, , , , , , HO KIONG CHAN]
2           [, , , , , ,  AHMAD ASABI BIN ISMAIL]
3                       [, , , , , , ANG LEE ENG]
4                       [, , , , , , ANG LEE ENG]
                           ...                   
715256                  [, , , , , ,  MUHD UMAIR]
715257                  [, , , , , ,  MUHD UMAIR]
715258                  [, , , , , ,  MUHD UMAIR]
715259                [, , , , , , MARIKO SUZUKI]
715260    [LIUAYUAN YUAN, , , , , , YUANYUAN LIU]
Name: ALL_WL_NAMES, Length: 715261, dtype: object

In [302]:
df_to_match=pd.DataFrame({})
df_to_match["ALL_ALIAS_NAMES"]=name_df["ALL_ALIAS_NAMES"].values
df_to_match["ALL_PARTY_NAMES"]=name_df["ALL_PARTY_NAMES"].values
df_to_match["ALL_WL_NAMES"]=all_wl_nam["ALL_WL_NAMES"].values
df_to_match["V_COMMENTS"]=df["V_COMMENTS"].values
df_to_match["V_RESOLUTION"]=df["V_RESOLUTION"].values
df_to_match["N_MATCH_SCORE"]=df["N_MATCH_SCORE"].values

In [303]:
df_to_match

,ALL_ALIAS_NAMES,ALL_PARTY_NAMES,ALL_WL_NAMES,V_COMMENTS,V_RESOLUTION,N_MATCH_SCORE
0,"[, , , , , , , , , , , , , , , ]","[SAHARUDDIN BIN MD SALLEH, , , ]","[, , , , , , SHARUDDIN BIN MD SALLEH]",Mismatch ID number_NRIC\n,FP,95.0
1,"[, , , , , , , , , , , , , , , ]","[HO KIONG CHAN, , , ]","[, , , , , , HO KIONG CHAN]",,TH,100.0
2,"[, , , , , , , , , , , , , , , ]","[ AHMAD ASABI BIN ISMAIL, , , ]","[, , , , , , AHMAD ASABI BIN ISMAIL]",,TH,100.0
3,"[, , , , , , , , , , , , , , , ]","[ANG LEE ENG, , , ]","[, , , , , , ANG LEE ENG]",Mismatch ID number_NRIC,FP,100.0
4,"[, , , , , , , , , , , , , , , ]","[ENG LEE ANG, , , ]","[, , , , , , ANG LEE ENG]",Mismatch ID Number_NRIC,FP,100.0
...,...,...,...,...,...,...
715256,"[, , , , , , , , , , , , , , , ]","[ MUHD UMAR, , , ]","[, , , , , , MUHD UMAIR]",Mismatch Full Legal Name – Name Does Not Sound Similar\n,FP,90.0
715257,"[, , , , , , , , , , , , , , , ]","[ MUHD UMAR, , , ]","[, , , , , , MUHD UMAIR]",Mismatch Father's Name - Difference in Father's Name. Comparison was made between customer’s off...,FP,90.0
715258,"[, , , , , , , , , , , , , , , ]","[ MUHD UMAIR, , , ]","[, , , , , , MUHD UMAIR]",Mismatch Father's Name - Difference in Father's Name\nCustomer (EDMS): IQBAL\nWatchperson (World...,FP,100.0
715259,"[, , , , , , , , , , , , , , , ]","[MAIKO SUZUKI, , , ]","[, , , , , , MARIKO SUZUKI]",Mismatch Full Legal Name_Name Does Not Sound Similar,FP,93.0


In [304]:
import os
# Recipe inputs
folder_path = dataiku.Folder("Name Screening notebooks").get_path()
path_of_csv = os.path.join(folder_path, "Processed_Names.csv") #/* dataset_01.csv is one of the files in this folder */
df_to_match.to_csv(path_of_csv)


# EDA 

In [305]:
# Example: load a DSS dataset as a Pandas dataframe
mydataset = dataiku.Dataset("comb_data_ind_final3")
df = mydataset.get_dataframe()

/app/dataiku/design/code-envs/python/py_39_tm_ns_1/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3343: DtypeWarning: Columns (25,33,47,52,53,58,80,81,82,83,84) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [306]:
my_dataset = dataiku.Dataset("Standradization_rules_draft__base_line_model")
scr_rules = my_dataset.get_dataframe()
scr_rules_2=scr_rules.iloc[21:45,:] # Punctuation marks
print(scr_rules.loc[46:51,"V_SCREENING_VALUE_1"])
#scr_rules.loc[46:51,"V_SCREENING_VALUE_1"]=scr_rules.loc[46:51,"V_SCREENING_VALUE_1"].apply(lambda x: x.lstrip("( ").rstrip(")"))
scr_rules_3=scr_rules.iloc[46:52,:]  #  Standardization rules
scr_rules=scr_rules.iloc[0:16,:]    # patronym & honourary titles

46     MOHAMMAD |MOHAMMED |MOHAMAD |MOHAMED | MOHD 
47    ABDUL |ABDOUL |ABDUR |ABDOL |ABDOOL |ABD |AB 
48                            NOOR |NOR |NUUR |NUR 
49                             AHMAD |AHMED |AHMET 
50                              ABDULLAH |ABDOLLAH 
51                    YUSOFF |YUSOF |YUSUFF |YUSUF 
Name: V_SCREENING_VALUE_1, dtype: object


In [307]:
scr_rules_3["V_SCREENING_VALUE_1"]

46     MOHAMMAD |MOHAMMED |MOHAMAD |MOHAMED | MOHD 
47    ABDUL |ABDOUL |ABDUR |ABDOL |ABDOOL |ABD |AB 
48                            NOOR |NOR |NUUR |NUR 
49                             AHMAD |AHMED |AHMET 
50                              ABDULLAH |ABDOLLAH 
51                    YUSOFF |YUSOF |YUSUFF |YUSUF 
Name: V_SCREENING_VALUE_1, dtype: object

In [308]:
scr_rules

,Sequence of the rules,col_1,V_SCREENING_VALUE_1,Remarks,Solution,V_SCREENING_VALUE_2,col_6
0,1,Patronym,A/L | A/P | S/O | D/O | SO | DO | AP,Terms are used to signify the father's name like A/P refers to Daughter of. For eg. VERNISHA J...,We remove these these terms,NaN,NaN
1,1,Patronym,BINTI | BINTE | B | BTE | BT | IBNI | IBNU | ANAK,Son / daughter of. for Malay & indegenous people. For eg. FARIDAH BINTE MAHMUD,We remove these these terms,NaN,NaN
2,2,Apostrophe,\',Change in sound,Remove it,NaN,NaN
3,3,Hyphen,[-],use to join words,Replace with white space character,NaN,NaN
4,4,Prefix,HAJI |HAJJAH |HAJAH |HJH |HJ |ALHAJ |HADJI,Tilte for˜Muslim˜men and women who have performed the˜Hajj (pilgrimage). That means these terms ...,Remove the terms if present in Individual( also in father's name if it comes),NaN,NaN
5,4,Prefix,WIRA |PANGLIMA |IR |DR. |PROF. |SR,Other salutations; professional rank,Remove the salutations; professional rank,NaN,NaN
6,5,Titles for Royal and honoured people,K.D.Y.T.M. |K.D.Y.M.M. |D.Y.M.M. |D.Y.T.M. |Y.T.M. |D.Y.A.M. |Y.M.M. |Y.A.M. |Y.BHG. |KDYTM |KDY...,Abbreviation for the king title,Remove the title,NaN,NaN
7,5,Titles for Royal and honoured people,YANG DI PERTUAN AGONG |DULI YANG MAHA MULIA SERI PADUKA BAGINDA,Official title of the king(ruler) of all Malaysia,Remove the title,NaN,NaN
8,5,Titles for Royal and honoured people,TIMBALAN YANG DI PERTUAN AGONG,NaN,Remove the title,NaN,NaN
9,5,Titles for Royal and honoured people,YANG DI PERTUAN BESAR |YANG DI PERTUA NEGERI |SULTAN,NaN,Remove the title,NaN,NaN


In [309]:
# pre-processing on "V_REGD_PARTY_NAME"
print(df["V_REGD_PARTY_NAME"].apply(lambda x:type(x)).unique())
df["V_REGD_PARTY_NAME"]=df["V_REGD_PARTY_NAME"].apply(lambda x : "" if pd.isna(x) else str(x).upper())

[<class 'str'>]


## Check for patronyms

In [310]:
# Patronym , ' - , Honourary Titles 
df[df["V_REGD_PARTY_NAME"].apply(lambda x: len(re.findall("HAJI |HAJJAH |HAJAH |HJH |HJ |ALHAJ |HADJI",x))>0)]["V_REGD_PARTY_NAME"]

df[df["V_REGD_PARTY_NAME"].apply(lambda x: len(re.findall(' A/L | A/P | S/O | D/O | SO | DO | AP | BINTI | BINTE | B | BTE | BT | IBNI | IBNU | ANAK ',x))>0)]["V_REGD_PARTY_NAME"]

12                 GANESAN S/O SUBRAMANIAM
13                 GANESAN A/L SUBRAMANIAM
14                 GANESAN A/L SUBRAMANIAM
15                 GANESON A/L SUBRAMANIAM
16                 GANESAN A/L SUBRAMANIAM
                        ...               
701045        NURBALQIS ADRIANA BINTI MOHD
701046           SITI KHADIJAH BINTI HARON
701108    ALEXANDER MAXWELL ANAK NGELAYANG
701120              ZURINAWATI BINTI AWANG
701135                 PAVITHRA A/P SEGREN
Name: V_REGD_PARTY_NAME, Length: 108141, dtype: object

In [311]:
df[df["V_PROFILE_NAME"].apply(lambda x: len(re.findall("HAJI |HAJJAH |HAJAH |HJH |HJ |ALHAJ |HADJI",x))>0)]["V_PROFILE_NAME"]

df[df["V_PROFILE_NAME"].apply(lambda x: len(re.findall(' A/L | A/P | S/O | D/O | SO | DO | AP | BINTI | BINTE | B | BTE | BT | IBNI | IBNU | ANAK ',x))>0)]["V_PROFILE_NAME"]

12                 GANESAN A/L SUBRAMANIAM
13                 GANESAN A/L SUBRAMANIAM
14                 GANESAN A/L SUBRAMANIAM
15                 GANESAN A/L SUBRAMANIAM
16                 GANESAN A/L SUBRAMANIAM
                        ...               
701045        NURBALQIS ADRIANA BINTI MOHD
701046           SITI KHADIJAH BINTI HARON
701108    ALEXANDER MAXWELL ANAK NGELAYANG
701120              ZURINAWATI BINTI AWANG
701135                 PAVITHRA A/P SEGREN
Name: V_PROFILE_NAME, Length: 69532, dtype: object

In [312]:
print(df[df["V_REGD_PARTY_NAME"].apply(lambda x: len(re.findall("\'|-",x[0]))>0)]["V_REGD_PARTY_NAME"])
df[df["V_PROFILE_NAME"].apply(lambda x: len(re.findall("\'|-",x[0]))>0)]["V_PROFILE_NAME"]

27226                          'AMMAR BIN ISMAIL
53639                           'AZIZI BIN ISHAK
71352                     'AMIRAH BINTI ABDULLAH
89531                   'ATIQAH BINTI MOHD YUSOF
100317               'ATIQAH BINTI ZAINAL ABIDIN
102053      'ABDUL MUNA'EEM BIN AHMAD SHAHBUDDIN
148825           ' MOHD YUSOFF BIN HAJI MD NASIR
151188                    'AIZUDDIN BIN MUSTAFFA
159330                    'ATIKAH BINTI ABDULLAH
210216                    'AIZUDDIN BIN MUSTAFFA
213685                      'AMIRAH BINTI ISMAIL
213691                      'AMIRAH BINTI ISMAIL
258681                                     'IMAH
264439           ' MOHD YUSOFF BIN HAJI MD NASIR
269115                    'ADILAH BINTI ABDULLAH
278556     'AZIZU HAKIM BIN MD ADAM @ MOHD ADNAN
293589                          'IMRAN BIN AHMAD
331338                        'ATIQAH BINTI MOHD
331360                     'ATIQAH BINTI MOHAMED
331365                     'ATIQAH BINTI MOHAMED
332167              

102053    'ABDUL MUNA'EEM BIN AHMAD SHAHBUDDIN
210215                  'AIZUDDIN BIN MUSTAFFA
210216                  'AIZUDDIN BIN MUSTAFFA
341449        'AINAA' NAJIHAH BINTI ABDUL AZIZ
Name: V_PROFILE_NAME, dtype: object

## Check for Honorary Titles

In [313]:
df[df["V_REGD_PARTY_NAME"].str.contains('RAJA |TENGKU |MEK |NIK |TUAN |WAN |PUTERI |PUTRI |AWANG |ABANG |DAYANG |CIK PUAN BESAR ')]["V_REGD_PARTY_NAME"]

83                  RAJA GANESAN A/L SUBRAMANIAM
243       RAJA NURUL ASHIKIN BINTI RAJA HANAFIAH
291              NIK BADROL ROSHEDAM BIN BAHARIM
345                        MUHD RIDWAN BIN WAHAP
394                              CHOONG WAN KHIM
                           ...                  
700480              MUHAMMAD RIDWAN BIN ABDULLAH
700640              WAN NORAZLINA BINTI WAN DAUD
700782               AWANG AHMAD BIN AWANG BAKAR
700960                 RAJA ZAHAR BIN RAJA AHMAD
701028              MOHD SAPUWAN BIN MOHD RAZALI
Name: V_REGD_PARTY_NAME, Length: 11938, dtype: object

In [314]:
df[df["V_PROFILE_NAME"].str.contains('RAJA |TENGKU |MEK |NIK |TUAN |WAN |PUTERI |PUTRI |AWANG |ABANG |DAYANG |CIK PUAN BESAR ')]["V_PROFILE_NAME"]

243       RAJA NURUL ASHIKIN BINTI RAJA HANAFIAH
291              NIK BADROL ROSHEDAM BIN BAHARIM
345                        MUHD RIDWAN BIN WAHAP
394                              CHOONG WAN KHIM
457              NIK NUR ASMARA AMIRA BINTI AMIR
                           ...                  
700636         TUAN NOR SYAHIDA BINTI TUAN AHMAD
700637         TUAN NOR SYAHIDA BINTI TUAN AHMAD
700638         TUAN NOR SYAHIDA BINTI TUAN AHMAD
700640              WAN NORAZLINA BINTI WAN DAUD
701028              MOHD SAPUWAN BIN MOHD RAZALI
Name: V_PROFILE_NAME, Length: 9764, dtype: object

## Check for Alias Names

In [315]:
print(df[df["V_REGD_PARTY_NAME"].apply(lambda x: len(re.findall("@",x[0]))>0)]["V_REGD_PARTY_NAME"])

print(df[df["V_REGD_PARTY_NAME"].apply(lambda x: len(re.findall(",",x[0]))>0)]["V_REGD_PARTY_NAME"])

print(df[df["V_REGD_PARTY_NAME"].apply(lambda x: len(re.findall("\(.+\)",x))>0)]["V_REGD_PARTY_NAME"])

<>:5: DeprecationWarning: invalid escape sequence \(
<>:5: DeprecationWarning: invalid escape sequence \(
<ipython-input-315-f8e6bfc17b9d>:5: DeprecationWarning: invalid escape sequence \(
  print(df[df["V_REGD_PARTY_NAME"].apply(lambda x: len(re.findall("\(.+\)",x))>0)]["V_REGD_PARTY_NAME"])


Series([], Name: V_REGD_PARTY_NAME, dtype: object)
Series([], Name: V_REGD_PARTY_NAME, dtype: object)
45360                SOON (SOIN) CHIN WEI
75686                 NGE (NG) JIAN XIANG
129759                  LEE WAN TIN (DR.)
152088        FATIMAH (HJH) BT HJ IBRAHIM
166782         YEO KAY HIAN (YANG QIXIAN)
188081          WAN IBRAHIM WAN HASSAN(A)
193156            MOHAMAD BIN MUSA (HAJI)
312197    MOHAMMAD REZUANUL HOQUE (JEWEL)
333852            MOHAMAD BIN MUSA (HAJI)
447902        SEAH KOK PENG (SHE GUOPING)
480048            MOHAMAD BIN MUSA (HAJI)
506078          ZAIPAH BINTI YUSOF (MOHD)
516121    SEOW CHOON CHOON(XIAO CHUNCHUN)
601150                  LEE WAN TIN (DR.)
669025                  LEE WAN TIN (DR.)
676088             OOI YEOW TATT(BERNARD)
Name: V_REGD_PARTY_NAME, dtype: object


In [316]:
print(df[df["V_PROFILE_NAME"].apply(lambda x: len(re.findall("@",x[0]))>0)]["V_PROFILE_NAME"])

print(df[df["V_PROFILE_NAME"].apply(lambda x: len(re.findall(",",x[0]))>0)]["V_PROFILE_NAME"])

print(df[df["V_PROFILE_NAME"].apply(lambda x: len(re.findall("\(.+\)",x))>0)]["V_PROFILE_NAME"])

<>:5: DeprecationWarning: invalid escape sequence \(
<>:5: DeprecationWarning: invalid escape sequence \(
<ipython-input-316-2c52105df09d>:5: DeprecationWarning: invalid escape sequence \(
  print(df[df["V_PROFILE_NAME"].apply(lambda x: len(re.findall("\(.+\)",x))>0)]["V_PROFILE_NAME"])


Series([], Name: V_PROFILE_NAME, dtype: object)
Series([], Name: V_PROFILE_NAME, dtype: object)
45360                SOON (SOIN) CHIN WEI
62689        BARRY TAN CHONG LIANG (DATO)
75686                 NGE (NG) JIAN XIANG
127626              KER KIM LOONG (DATO')
166782         YEO KAY HIAN (YANG QIXIAN)
312197    MOHAMMAD REZUANUL HOQUE (JEWEL)
447902        SEAH KOK PENG (SHE GUOPING)
516121    SEOW CHOON CHOON(XIAO CHUNCHUN)
676088             OOI YEOW TATT(BERNARD)
Name: V_PROFILE_NAME, dtype: object


## Check for Accented characters

In [317]:
df[df["V_REGD_PARTY_NAME"].str.contains(r"[Á,Ä,Â,Ã]")]["V_REGD_PARTY_NAME"]
df[df["V_REGD_PARTY_NAME"].str.contains("[Å]")]["V_REGD_PARTY_NAME"]
df[df["V_REGD_PARTY_NAME"].str.contains("[É,Ë,Ê,È]")]["V_REGD_PARTY_NAME"]

2316      ZHANG, HONGYING
2320          ZHAN, PUWEN
2328             LIU, JIA
2483          ZHANG, MENG
2485          ZHANG, PENG
               ...       
701128         WANG, QIAN
701134      LI, XIANGGANG
701141         ZHANG, YAN
701143        ZHANG, YING
701180       LU, YUANYUAN
Name: V_REGD_PARTY_NAME, Length: 6100, dtype: object

In [318]:
df[df["V_REGD_PARTY_NAME"].str.contains("[Á,Ä,Â,Ã]")]["V_REGD_PARTY_NAME"]
df[df["V_REGD_PARTY_NAME"].str.contains("[Å]")]["V_REGD_PARTY_NAME"]
df[df["V_REGD_PARTY_NAME"].str.contains("[É,Ë,Ê,È]")]["V_REGD_PARTY_NAME"]
df[df["V_REGD_PARTY_NAME"].str.contains("[Í,Ï,Î]","I")]["V_REGD_PARTY_NAME"]
df[df["V_REGD_PARTY_NAME"].str.contains("[Ó,Ô,Ö,Õ]")]["V_REGD_PARTY_NAME"]
df[df["V_REGD_PARTY_NAME"].str.contains("[Ú,Ü,Ù]")]["V_REGD_PARTY_NAME"]

df[df["V_REGD_PARTY_NAME"].str.contains("[Ø]")]["V_REGD_PARTY_NAME"]
df[df["V_REGD_PARTY_NAME"].str.contains("[Ç]")]["V_REGD_PARTY_NAME"]
df[df["V_REGD_PARTY_NAME"].str.contains("[Ñ]")]["V_REGD_PARTY_NAME"]
df[df["V_REGD_PARTY_NAME"].str.contains("[Ý]")]["V_REGD_PARTY_NAME"]
df[df["V_PROFILE_NAME"].str.contains("[Á,Ä,Â,Ã,É,Ë,Ê,È,Å,Í,Ï,Î,Ó,Ô,Ö,Õ,Ú,Ü,Ù,Ø,Ý,Ñ]")]["V_PROFILE_NAME"]                                                                                     

4246                    Tet Hau,NG
4247                    Tet Hau,NG
4248              Van Cuong,NGUYEN
4249              Van Cuong,NGUYEN
21893                Kuen Yin,WONG
                    ...           
675635    KUMARAGURU THANASEKARAN,
679542        Justin Yung Bin,WONG
680270                HU, JIANPING
684977                 LIU, ZHENYU
685302               ZHAO, JINGHUI
Name: V_PROFILE_NAME, Length: 480, dtype: object

## Check for Punctuation Marks

In [319]:
# check under REGD_PARTY_NAMES for punctuations

print(df[df["V_REGD_PARTY_NAME"].apply(lambda x: len(re.findall("\{.*\}|\[.*\]|\\\\|:|;|\"|<|>|\?",x[0]))>0)]["V_REGD_PARTY_NAME"])
print(df[df["V_PROFILE_NAME"].apply(lambda x: len(re.findall("\{.*\}|\[.*\]|\\\\|:|;|\"|<|>|\?",x[0]))>0)]["V_PROFILE_NAME"])


<>:3: DeprecationWarning: invalid escape sequence \{
<>:4: DeprecationWarning: invalid escape sequence \{
<>:3: DeprecationWarning: invalid escape sequence \{
<>:4: DeprecationWarning: invalid escape sequence \{
<ipython-input-319-c156e5a53cc2>:3: DeprecationWarning: invalid escape sequence \{
  print(df[df["V_REGD_PARTY_NAME"].apply(lambda x: len(re.findall("\{.*\}|\[.*\]|\\\\|:|;|\"|<|>|\?",x[0]))>0)]["V_REGD_PARTY_NAME"])
<ipython-input-319-c156e5a53cc2>:4: DeprecationWarning: invalid escape sequence \{
  print(df[df["V_PROFILE_NAME"].apply(lambda x: len(re.findall("\{.*\}|\[.*\]|\\\\|:|;|\"|<|>|\?",x[0]))>0)]["V_PROFILE_NAME"])


Series([], Name: V_REGD_PARTY_NAME, dtype: object)
Series([], Name: V_PROFILE_NAME, dtype: object)


## Check for Standardized names

In [320]:
for k in scr_rules_3[["V_SCREENING_VALUE_1","Solution"]].values:
    print(df[df["V_REGD_PARTY_NAME"].str.contains(k[0])]["V_REGD_PARTY_NAME"].head())
    
for k in scr_rules_3[["V_SCREENING_VALUE_1","Solution"]].values:
    print(df[df["V_PROFILE_NAME"].str.contains(k[0])]["V_PROFILE_NAME"].head())

235           MOHAMAD IQBAL BIN LAHBUDIN
284          NUR IZZATI BT MOHAMAD RODZI
285          NUR IZZATI BINTI MOHD RADZI
288    MOHAMMAD AFIQ IZZUDDIN BIN ZAINOL
305            AHMAD HUSNI BIN MOHD NAWI
Name: V_REGD_PARTY_NAME, dtype: object
262       SALLEH BIN ABDUL SALLEH
273      AZWANI BINTI ABDUL JALIL
274    JAMALIAH BINTI ABDUL GHANI
275          KAMALIAH BT AB GHANI
276    KAMARIAH BINTI ABDUL GHANI
Name: V_REGD_PARTY_NAME, dtype: object
284    NUR IZZATI BT MOHAMAD RODZI
285    NUR IZZATI BINTI MOHD RADZI
379              MOHD NOOR BIN ABI
425       NUR ASHIKIN BINTI BAHARI
426      NUR ASHIKIN BINTI BAHARUM
Name: V_REGD_PARTY_NAME, dtype: object
2                      AHMAD ASABI BIN ISMAIL
305                 AHMAD HUSNI BIN MOHD NAWI
375                    AHMAD BIN HJ IBNIHAJAR
586    SITI INTAN SURAYA BINTI AHMAD SOHEIMIN
635                AHMAD FARID BIN SHAHARUDIN
Name: V_REGD_PARTY_NAME, dtype: object
710               ABDULLAH BIN MOHAMAD
712               ABDULLA

## Check for miscellaneous characters

In [321]:
# For any other miscellaneous character :Won't be able to do that because the above speacial characters have not been removed.
df["V_REGD_PARTY_NAME"].str.contains("[^A-Z\s ]+")
df["V_PROFILE_NAME"].str.contains("[^A-Z\s ]+")

<>:2: DeprecationWarning: invalid escape sequence \s
<>:3: DeprecationWarning: invalid escape sequence \s
<>:2: DeprecationWarning: invalid escape sequence \s
<>:3: DeprecationWarning: invalid escape sequence \s
<ipython-input-321-44cdd51f8b52>:2: DeprecationWarning: invalid escape sequence \s
  df["V_REGD_PARTY_NAME"].str.contains("[^A-Z\s ]+")
<ipython-input-321-44cdd51f8b52>:3: DeprecationWarning: invalid escape sequence \s
  df["V_PROFILE_NAME"].str.contains("[^A-Z\s ]+")


0         False
1         False
2         False
3         False
4         False
          ...  
701176     True
701177     True
701178     True
701179     True
701180     True
Name: V_PROFILE_NAME, Length: 701181, dtype: bool

## Comments Analysis

In [322]:
df["V_COMMENTS"]=df["V_COMMENTS"].apply(lambda x: "" if pd.isna(x) else x)
# We assume that all the alerts that are close because of names anchor should have name as text in comments .Now secondly, 
# we need to check that when commenst contain name as string , whether the context of comment is alogned with resolution TH , THNM, FP.
# So we obtain value counts for TH, THNM & FP .Now check with existing observed comments like Match Full leagal name , to see what extent to we approach to these value counts.
patt=re.compile("name",re.IGNORECASE)
print(df[df["V_COMMENTS"].str.contains(patt,re.IGNORECASE)].shape)
print(df[df["V_COMMENTS"].str.contains(patt,re.IGNORECASE)]["V_RESOLUTION"].value_counts())

(194968, 112)
FP      127671
TH       65590
THNM      1707
Name: V_RESOLUTION, dtype: int64


### Alerts with resolution TH,THNM and comment has name match

In [323]:
patt_2=re.compile("Match Full Legal Name|Match Name|Match_ Full Legal Name|Match name|Match_name|Matched Name|Matched name\
Match_Name|Match_Full Legal Name|Match_ Name|Match Full Legal  Name |Match ID and Full Legal Name|Match ID or Full Legal Name|Match ID & Legal Name|Match ID and/or Full Legal Name",re.IGNORECASE)
df_comments_TH=df[(df["V_COMMENTS"].str.contains(patt_2)) & (df["V_RESOLUTION"].isin(["TH","THNM"]))]
df[(df["V_COMMENTS"].str.contains(patt_2)) & (df["V_RESOLUTION"].isin(["TH","THNM"]))].shape

(62596, 112)

In [324]:
print(df[(df["V_COMMENTS"].str.contains(patt)) & (~df["V_COMMENTS"].str.contains(patt_2)) & (df["V_RESOLUTION"].isin(["TH","THNM"]))]["V_COMMENTS"].shape)
df[(df["V_COMMENTS"].str.contains(patt)) & (~df["V_COMMENTS"].str.contains(patt_2)) & (df["V_RESOLUTION"].isin(["TH","THNM"]))]["V_COMMENTS"].value_counts()

(4701,)


Match ID number_NRIC, Date of Birth (DOB) & Full Legal Name                                                                                                                                                                                                                                                                                        176
Match ID & Full Legal Name                                                                                                                                                                                                                                                                                                                         112
Match ID number_NRIC & Full Legal name\n                                                                                                                                                                                                                                                                                  

In [325]:
# following are the alerts with resolution TH,THNM and comment says match name but names have variations 
print("shape of df_comments_TH",df_comments_TH.shape)
print("count of alerts with name variations",df_comments_TH[df_comments_TH["V_REGD_PARTY_NAME"]!=df_comments_TH["V_PROFILE_NAME"]].shape)
print("% count of alerts with name variations ",df_comments_TH[df_comments_TH["V_REGD_PARTY_NAME"]!=df_comments_TH["V_PROFILE_NAME"]].shape[0]/df_comments_TH.shape[0]*100 )
df_comments_TH[df_comments_TH["V_REGD_PARTY_NAME"]!=df_comments_TH["V_PROFILE_NAME"]][["V_REGD_PARTY_NAME","V_PROFILE_NAME","V_RESOLUTION"]]

shape of df_comments_TH (62596, 112)
count of alerts with name variations (3973, 112)
% count of alerts with name variations  6.3470509297718705


,V_REGD_PARTY_NAME,V_PROFILE_NAME,V_RESOLUTION
882,RADEN MOHAMMAD SHAFIQ BIN RM EFFENDI HUS,RADEN MOHAMMAD SHAFIQ BIN RM EFFENDI HUSIN,TH
1490,ABD ALRAHMAN KHER ALLAH,Abd Alrahman Kher Allah,TH
1498,AHMAD AMRO JANO,Ahmad Amro Jano,TH
1499,HISHAM ALMEDANI,Hisham Almedani,TH
1577,MOHD NURIZZUDDIN AZRAAI,MOHD NURIZZUDDIN AZRA'AI BIN AWANG,TH
...,...,...,...
699358,NORWANI BINTI MUHAMAD SHARIFF,Norwani Binti MOHD SHARIFF,THNM
699609,KATIMI BIN AZMI,Katimi AZMI,TH
699717,NURUL HAZLIN BINTI HASHIM,Nurul Hazlin HASHIM,TH
700027,AIDA AHMED HAMOOD AL AISARI,Aida Ahmed Hamood AL-AISARI,TH


### Comments with name miss-match & resolution FP

In [326]:
print(df[(df["V_COMMENTS"].str.contains(patt)) & (df["V_RESOLUTION"]=="FP")].shape)
df[(df["V_COMMENTS"].str.contains(patt)) & (df["V_RESOLUTION"]=="FP")]["V_COMMENTS"].value_counts()

(127671, 112)


Mismatch Full Legal Name_Name Does Not Sound Similar                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       15124
Mismatch Full Legal Name_Name Does Not Sound Similar\n                 

In [327]:
# We need to check whether the context in which name is coming in comments is aligned with resolution FP.So we have existing 
# comments based on which we validate that upto how much alerts with name in comments & FP can we achieve.
patt_3=re.compile("Mismatch Full Legal Name|sound",re.IGNORECASE)
df_comments_FP=df[(df["V_COMMENTS"].str.contains(patt_3)) & (df["V_RESOLUTION"]=="FP")]

df[(df["V_COMMENTS"].str.contains(patt_3)) & (df["V_RESOLUTION"]=="FP")]["V_COMMENTS"].shape


(113098,)

In [328]:
ind=df[ (df["V_COMMENTS"].str.contains(patt)) & (df["V_RESOLUTION"]=="FP")].index
ind_2=df[~(df["V_COMMENTS"].str.contains(patt_3))].index
print(df.loc[list(set(ind).intersection(set(ind_2))),"V_COMMENTS"].shape)
#df.loc[list(set(ind).intersection(set(ind_2))),"V_COMMENTS"].value_counts()

# We see that those alerts having name in comments and resolution FP are not present in correct context as per what is required to select
# the alerts closed due to name anchor as they contain father's name in comments.Their total count is 14,575 which is the difference 1.27 lac and 1.13 Lac So we take pre-defined comments that amounts to 1,13,098.

(14573,)


In [329]:
df_comments_FP[["V_REGD_PARTY_NAME","V_PROFILE_NAME","V_RESOLUTION"]]

,V_REGD_PARTY_NAME,V_PROFILE_NAME,V_RESOLUTION
114,KANESAN A/L SUBRAMANIAM,GANESAN A/L SUBRAMANIAM,FP
249,RAZALI BIN AHMAD,ROZALI BIN AHMAD,FP
299,TAN HOCK BENG,TAN HOCK HENG,FP
454,SALINA BINTI MOHD YUSOF,SHARLINA BINTI MOHD YUSOFF,FP
462,OOI LEONG KUAN,OOI KEAN LEONG,FP
...,...,...,...
701170,ABDUL ALIM BIN ABDULLAH,ABDUL RAHIM BIN ABDULLAH,FP
701172,ABDUL KARIM BIN ABDULLAH,ABDUL RAHIM BIN ABDULLAH,FP
701176,MUHAMMAD UMAR,Muhammad UMAIR,FP
701179,MAIKO SUZUKI,Mariko SUZUKI,FP


In [330]:
df_combined=pd.concat([df_comments_FP,df_comments_TH],axis=0,join="outer")

In [331]:
df_combined
# Need to drop the alerts where any of the name is empty string.
print(df_combined[ (df_combined["V_REGD_PARTY_NAME"].isin([""])) | (df_combined["V_PROFILE_NAME"].isin([""]))][["V_REGD_PARTY_NAME","V_PROFILE_NAME"]].shape)
df_combined.drop(df_combined[ (df_combined["V_REGD_PARTY_NAME"].isin([""])) | (df_combined["V_PROFILE_NAME"].isin([""]))].index,axis=0,inplace=True)

(0, 2)


## Check for duplicates

In [332]:

print("rows in combined data {:.2f}".format(df_combined.shape[0]))
print("counts rows that are present as duplicated based on names of customer & wl",df_combined[df_combined.duplicated(subset=(["V_REGD_PARTY_NAME","V_PROFILE_NAME","V_RESOLUTION"]))].shape)
df_combined.drop_duplicates(subset=(["V_REGD_PARTY_NAME","V_PROFILE_NAME","V_RESOLUTION"]),inplace=True)
print("count of alerts without duplicates {:.2f}".format(df_combined.shape[0]))

rows in combined data 175694.00
counts rows that are present as duplicated based on names of customer & wl (79926, 112)
count of alerts without duplicates 95768.00


# Name Matching

In [333]:
from Levenshtein import distance

In [334]:
import unidecode
from fuzzywuzzy import fuzz

#import syllable_tokenizer

from abydos.distance import (IterativeSubString, BISIM, DiscountedLevenshtein, Prefix, LCSstr, MLIPNS, Strcmp95,
MRA, Editex, SAPS, FlexMetric, JaroWinkler, HigueraMico, Sift4, Eudex, ALINE, Covington, PhoneticEditDistance)

from abydos.phonetic import PSHPSoundexFirst, Ainsworth
pshp_soundex_first = PSHPSoundexFirst()
pe = Ainsworth()

In [335]:
iss = IterativeSubString()
bisim = BISIM()
lev = Levenshtein()
prefix = Prefix()
lcs = LCSstr()
mlipns = MLIPNS()
strcmp95 = Strcmp95()
mra = MRA()
editex = Editex()
saps = SAPS()
flexmetric = FlexMetric()
jaro = JaroWinkler(mode='Jaro')
higuera_mico = HigueraMico()
sift4 = Sift4()
eudex = Eudex()
aline = ALINE()
covington = Covington()
phonetic_edit = PhoneticEditDistance()


NameError: name 'Levenshtein' is not defined